In [1]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
from scipy.stats import mode
import csv
import matplotlib.dates
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
from datetime import *
from datetime import  datetime
import matplotlib.pylab as pylab
from pylab import *  
mpl.rcParams['font.sans-serif'] = ['SimHei']

In [58]:
#首先导入用户基本信息，作为每个月的样本，构造训练集和测试集
#用户基本信息表--jdata_user_basic_info
jdata_user_basic_info_data = pd.read_csv('../../data/jdata_user_basic_info.csv')
print(jdata_user_basic_info_data.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98924 entries, 0 to 98923
Data columns (total 4 columns):
user_id       98924 non-null int64
age           98924 non-null int64
sex           98924 non-null int64
user_lv_cd    98924 non-null int64
dtypes: int64(4)
memory usage: 3.0 MB
None


In [59]:
#用户订单表--jdata_user_order
jdata_user_order_data = pd.read_csv('../../data/jdata_user_order.csv')
print(jdata_user_order_data.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 792723 entries, 0 to 792722
Data columns (total 6 columns):
user_id      792723 non-null int64
sku_id       792723 non-null int64
o_id         792723 non-null int64
o_date       792723 non-null object
o_area       792723 non-null int64
o_sku_num    792723 non-null int64
dtypes: int64(5), object(1)
memory usage: 36.3+ MB
None


In [60]:
#导入剩余的3张表
#用户行为表--jdata_user_action
jdata_user_action_data = pd.read_csv('../../data/jdata_user_action.csv')
print(jdata_user_action_data.info())

#SKU基本信息表--jdata_sku_basic_info
jdata_sku_basic_info_data = pd.read_csv('../../data/jdata_sku_basic_info.csv')
print(jdata_sku_basic_info_data.info())

#评论分数数据表--jdata_user_comment_score
jdata_user_comment_score_data = pd.read_csv('../../data/jdata_user_comment_score.csv')
print(jdata_user_comment_score_data.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6944141 entries, 0 to 6944140
Data columns (total 5 columns):
user_id    int64
sku_id     int64
a_date     object
a_num      int64
a_type     int64
dtypes: int64(4), object(1)
memory usage: 264.9+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99412 entries, 0 to 99411
Data columns (total 6 columns):
sku_id    99412 non-null int64
price     99412 non-null float64
cate      99412 non-null int64
para_1    99412 non-null float64
para_2    99412 non-null int64
para_3    99412 non-null int64
dtypes: float64(2), int64(4)
memory usage: 4.6 MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 224284 entries, 0 to 224283
Data columns (total 4 columns):
user_id              224284 non-null int64
comment_create_tm    224284 non-null object
o_id                 224284 non-null int64
score_level          224284 non-null int64
dtypes: int64(3), object(1)
memory usage: 6.8+ MB
None


In [5]:
#首先进行数据清洗操作，对用户行为表中同一天用户的某个订单中出现同样商品的两条样本进行数量上的相加操作
print(len(jdata_user_order_data))
jdata_user_order_data = pd.pivot_table(jdata_user_order_data, index=['user_id', 'o_id', 'sku_id', 'o_date', 'o_area'], values=['o_sku_num'], aggfunc=sum)
jdata_user_order_data.reset_index(inplace=True)
print(len(jdata_user_order_data))


792723
790923


In [6]:
#定义时间处理函数
def dealDateColumns(df, columnName):
    df[columnName] = pd.to_datetime(df[columnName])
    df['year'] = df[columnName].map(lambda x: x.year)
    df['month'] = df[columnName].map(lambda x: x.month)
    df['day'] = df[columnName].map(lambda x: x.day)
    return df

jdata_user_order_data = dealDateColumns(jdata_user_order_data, 'o_date')
jdata_user_action_data = dealDateColumns(jdata_user_action_data, 'a_date')
jdata_user_comment_score_data = dealDateColumns(jdata_user_comment_score_data, 'comment_create_tm')



In [7]:
print(len(set(jdata_user_comment_score_data['user_id'])))

42191


In [8]:
#对评分分数数据表中短时间内评分次数超过订单中商品个数的样本进行过滤操作
jdata_user_order_data_pivot_table = pd.pivot_table(jdata_user_order_data, index=['user_id', 'o_id'], values=['sku_id'], aggfunc=len)
jdata_user_order_data_pivot_table.reset_index(inplace=True)

jdata_user_comment_score_data_pivot_table = pd.pivot_table(jdata_user_comment_score_data, index=['user_id', 'o_id'], values=['score_level'], aggfunc=len)
jdata_user_comment_score_data_pivot_table.reset_index(inplace=True)
jdata_user_comment_score_data_pivot_table = pd.merge(jdata_user_comment_score_data_pivot_table, jdata_user_order_data_pivot_table, on=['user_id', 'o_id'], how='left')
jdata_user_comment_score_data_pivot_table['order_comment_diff'] = jdata_user_comment_score_data_pivot_table['sku_id'] - jdata_user_comment_score_data_pivot_table['score_level']

jdata_user_comment_score_data_zhuijia = jdata_user_comment_score_data_pivot_table[jdata_user_comment_score_data_pivot_table.order_comment_diff < 0]
jdata_user_comment_score_data_zhuijia = pd.merge(jdata_user_comment_score_data_zhuijia[['user_id', 'o_id', 'sku_id', 'order_comment_diff', 'score_level']], jdata_user_comment_score_data[['user_id', 'comment_create_tm', 'o_id']], on=['user_id', 'o_id'], how='left')
jdata_user_comment_score_data_zhuijia.rename(columns={'sku_id':'sku_num', 'score_level':'comment_num'}, inplace=True)

print(jdata_user_comment_score_data_zhuijia.head(10))

jdata_user_comment_score_data_zhuijia = jdata_user_comment_score_data_zhuijia.sort_index(by=['o_id', 'comment_create_tm'], ascending=True)
print(jdata_user_comment_score_data_zhuijia.head(10))

print(len(set(jdata_user_comment_score_data_pivot_table['user_id'])))
print(len(set(jdata_user_comment_score_data_pivot_table['user_id'][jdata_user_comment_score_data_pivot_table.order_comment_diff >= 0])))
print(len(set(jdata_user_comment_score_data_pivot_table['user_id'][jdata_user_comment_score_data_pivot_table.order_comment_diff < 0])))


   user_id    o_id  sku_num  order_comment_diff  comment_num  \
0     1710  273452        1                  -1            2   
1     1710  273452        1                  -1            2   
2     2088  163094        1                  -1            2   
3     2088  163094        1                  -1            2   
4     3471  158080        1                  -1            2   
5     3471  158080        1                  -1            2   
6     4156  447096        1                  -1            2   
7     4156  447096        1                  -1            2   
8     4156  649560        1                  -1            2   
9     4156  649560        1                  -1            2   

    comment_create_tm  
0 2016-10-14 12:48:59  
1 2016-10-14 12:49:12  
2 2017-02-14 16:01:27  
3 2017-02-14 16:02:22  
4 2016-10-15 07:22:16  
5 2016-10-15 07:21:38  
6 2016-10-22 15:43:47  
7 2016-11-13 13:36:51  
8 2016-10-04 14:46:11  
9 2016-10-18 12:24:20  
     user_id  o_id  sku_num  or

/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: FutureWarning: by argument to sort_index is deprecated, pls use .sort_values(by=...)
  app.launch_new_instance()


In [9]:
jdata_user_comment_score_data_zhuijia['last_comment_time'] = jdata_user_comment_score_data_zhuijia['comment_create_tm'].shift(1)
jdata_user_comment_score_data_zhuijia['last_comment_delta_time'] = jdata_user_comment_score_data_zhuijia['comment_create_tm'] - jdata_user_comment_score_data_zhuijia['last_comment_time']
jdata_user_comment_score_data_zhuijia['last_comment_delta_time'] = jdata_user_comment_score_data_zhuijia['last_comment_delta_time'].dt.seconds
jdata_user_comment_score_data_zhuijia['last_o_id'] = jdata_user_comment_score_data_zhuijia['o_id'].shift(1)
jdata_user_comment_score_data_zhuijia['same_o_id'] = jdata_user_comment_score_data_zhuijia['o_id'] == jdata_user_comment_score_data_zhuijia['last_o_id']

print(jdata_user_comment_score_data_zhuijia.info())

jdata_user_comment_score_data_zhuijia_left = pd.DataFrame(columns={'o_id', 'comment_create_tm'})
for o_id, comment_create_tm, same_o_id, last_comment_delta_time in jdata_user_comment_score_data_zhuijia[['o_id', 'comment_create_tm', 'same_o_id', 'last_comment_delta_time']].values:
    if ((same_o_id != True) | ((same_o_id == True) & (last_comment_delta_time > 180))):
        jdata_user_comment_score_data_zhuijia_left = jdata_user_comment_score_data_zhuijia_left.append({'o_id' : o_id, 'comment_create_tm' : comment_create_tm}, ignore_index=True)
        
print(jdata_user_comment_score_data_zhuijia_left.head(10)) 
print(len(jdata_user_comment_score_data_zhuijia_left))

jdata_user_comment_score_data_zhuijia_left = pd.merge(jdata_user_comment_score_data_zhuijia_left, jdata_user_comment_score_data, on=['o_id', 'comment_create_tm'], how='left')
print(len(jdata_user_comment_score_data_zhuijia_left))
print(jdata_user_comment_score_data_zhuijia_left.head(10))

print(len(set(jdata_user_comment_score_data_zhuijia_left['user_id'])))

jdata_user_comment_score_data_zhuijia_origion = jdata_user_comment_score_data_pivot_table[jdata_user_comment_score_data_pivot_table.order_comment_diff >= 0]
jdata_user_comment_score_data_zhuijia_origion = pd.merge(jdata_user_comment_score_data_zhuijia_origion[['user_id', 'o_id', 'sku_id', 'order_comment_diff', 'score_level']], jdata_user_comment_score_data[['user_id', 'comment_create_tm', 'o_id']], on=['user_id', 'o_id'], how='left')

print(len(jdata_user_comment_score_data_zhuijia_origion))
jdata_user_comment_score_data_zhuijia_origion.drop('score_level', axis=1, inplace=True)

jdata_user_comment_score_data_zhuijia_origion = pd.merge(jdata_user_comment_score_data_zhuijia_origion, jdata_user_comment_score_data, on=['user_id', 'o_id', 'comment_create_tm'], how='left')
jdata_user_comment_score_data_zhuijia_origion = jdata_user_comment_score_data_zhuijia_origion.drop_duplicates(['o_id', 'comment_create_tm', 'score_level'])
print(len(jdata_user_comment_score_data_zhuijia_origion))

print(len(jdata_user_comment_score_data))
jdata_user_comment_score_data = pd.concat([jdata_user_comment_score_data_zhuijia_left, jdata_user_comment_score_data_zhuijia_origion])
print(len(jdata_user_comment_score_data))

# jdata_user_comment_score_data_copy = jdata_user_comment_score_data.copy()
# print(len(jdata_user_comment_score_data_copy))
# jdata_user_comment_score_data_copy = jdata_user_comment_score_data_copy.drop_duplicates(['o_id', 'comment_create_tm'])
# print(len(jdata_user_comment_score_data_copy))




<class 'pandas.core.frame.DataFrame'>
Int64Index: 371 entries, 99 to 37
Data columns (total 10 columns):
user_id                    371 non-null int64
o_id                       371 non-null int64
sku_num                    371 non-null int64
order_comment_diff         371 non-null int64
comment_num                371 non-null int64
comment_create_tm          371 non-null datetime64[ns]
last_comment_time          370 non-null datetime64[ns]
last_comment_delta_time    370 non-null float64
last_o_id                  370 non-null float64
same_o_id                  371 non-null bool
dtypes: bool(1), datetime64[ns](2), float64(2), int64(5)
memory usage: 39.3 KB
None
    comment_create_tm   o_id
0 2017-02-04 08:49:10    443
1 2016-05-17 11:05:27   4648
2 2016-05-17 13:51:19   4648
3 2017-04-13 19:08:08   8996
4 2016-09-28 11:54:46   9099
5 2017-04-13 04:40:04   9975
6 2016-09-24 16:50:10  10881
7 2016-09-24 16:55:21  10881
8 2017-03-12 11:58:15  12542
9 2017-02-19 17:02:11  18448
202
202
   

In [10]:
#首先过滤出目标类目订单记录用于训练集打标签
jdata_user_order_data = pd.merge(jdata_user_order_data, jdata_sku_basic_info_data[['sku_id', 'cate']], on=['sku_id'], how='left')
jdata_user_order_data_target = jdata_user_order_data[(jdata_user_order_data.cate == 101) | (jdata_user_order_data.cate == 30)]
print(len(jdata_user_order_data))
print(len(jdata_user_order_data_target))


#首先尝试对每个月构造训练集，并为每条样本打上标签
def getTrainDfMonthly(jdata_train_df, jdata_user_basic_info_data, month, year):
    jdata_train_df_1 = jdata_user_basic_info_data.copy()
    jdata_train_df_1['month'] = month
    jdata_train_df_1['year'] = year
    jdata_train_df_1 = pd.merge(jdata_train_df_1, jdata_user_order_data_target[['user_id', 'year', 'month', 'day', 'o_date']], on=['user_id', 'year', 'month'], how='left')
    jdata_train_df_1 = jdata_train_df_1.sort_index(by=['user_id', 'o_date'], ascending=True)
    jdata_train_df_1 = jdata_train_df_1.drop_duplicates(['user_id'])
    jdata_train_df_1['o_date'] = jdata_train_df_1['o_date'].fillna(pd.to_datetime('2000-01-01'))
    jdata_train_df_1['is_order'] = jdata_train_df_1['day'].map(lambda x: 0 if math.isnan(x) else 1)
    jdata_train_df_1['order_day'] = jdata_train_df_1['day'].map(lambda x: -1 if math.isnan(x) else x)
    del jdata_train_df_1['day']
    del jdata_train_df_1['o_date']
    print(year, month, len(jdata_train_df_1[jdata_train_df_1.is_order == 1]))
    jdata_train_df = pd.concat([jdata_train_df, jdata_train_df_1])
    return jdata_train_df

jdata_train_df = pd.DataFrame(columns=['user_id', 'age', 'sex', 'user_lv_cd', 'month', 'year', 'is_order', 'order_day'])
jdata_train_df = getTrainDfMonthly(jdata_train_df, jdata_user_basic_info_data, 4, 2017)
jdata_train_df = getTrainDfMonthly(jdata_train_df, jdata_user_basic_info_data, 3, 2017)
jdata_train_df = getTrainDfMonthly(jdata_train_df, jdata_user_basic_info_data, 2, 2017)
jdata_train_df = getTrainDfMonthly(jdata_train_df, jdata_user_basic_info_data, 1, 2017)
jdata_train_df = getTrainDfMonthly(jdata_train_df, jdata_user_basic_info_data, 12, 2016)
jdata_train_df = getTrainDfMonthly(jdata_train_df, jdata_user_basic_info_data, 11, 2016)
jdata_train_df = getTrainDfMonthly(jdata_train_df, jdata_user_basic_info_data, 10, 2016)
jdata_train_df = getTrainDfMonthly(jdata_train_df, jdata_user_basic_info_data, 9, 2016)
jdata_train_df = getTrainDfMonthly(jdata_train_df, jdata_user_basic_info_data, 8, 2016)
jdata_train_df = getTrainDfMonthly(jdata_train_df, jdata_user_basic_info_data, 7, 2016)
jdata_train_df = getTrainDfMonthly(jdata_train_df, jdata_user_basic_info_data, 6, 2016)
# jdata_train_df = getTrainDfMonthly(jdata_train_df, jdata_user_basic_info_data, 5, 2016)
print(len(jdata_train_df))
print(jdata_train_df.head(10))


790923
549891
2017 4 53553


/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: FutureWarning: by argument to sort_index is deprecated, pls use .sort_values(by=...)
  


2017 3 52574
2017 2 47684
2017 1 31030
2016 12 28760
2016 11 31341
2016 10 25171
2016 9 20387
2016 8 17313
2016 7 14895
2016 6 17378
1088164
   user_id age sex user_lv_cd month  year is_order  order_day
1        1   3   2          3     4  2017        1       14.0
2        2   3   2          3     4  2017        1       24.0
3        3   3   0          5     4  2017        1       26.0
4        4   3   2          3     4  2017        1       22.0
5        5   4   2          2     4  2017        1       11.0
6        6  -1   2          2     4  2017        1       12.0
7        7   3   2          4     4  2017        1        7.0
8        8   2   2          2     4  2017        0       -1.0
9        9   3   2          3     4  2017        0       -1.0
10      10   4   0          4     4  2017        0       -1.0


In [11]:
#构造测试集
jdata_test_df = jdata_user_basic_info_data.copy()
jdata_test_df['year'] = 2017
jdata_test_df['month'] = 5
print(len(jdata_test_df))
print(jdata_test_df.head(10))


98924
   user_id  age  sex  user_lv_cd  year  month
0        1    3    2           3  2017      5
1        2    3    2           3  2017      5
2        3    3    0           5  2017      5
3        4    3    2           3  2017      5
4        5    4    2           2  2017      5
5        6   -1    2           2  2017      5
6        7    3    2           4  2017      5
7        8    2    2           2  2017      5
8        9    3    2           3  2017      5
9       10    4    0           4  2017      5


In [12]:
# 统计某用户距离上次购买相同类目商品的时间，逐个月进行处理
def getUserCateLastBuyDeltaTimeMonthly(jdata_user_basic_info_data, jdata_user_order_data, year, month, cate):
    jdata_train_df_1 = jdata_user_basic_info_data.copy()
    jdata_train_df_1['month'] = month
    jdata_train_df_1['year'] = year
    jdata_train_df_1['cate'] = cate
    time_boundary = datetime.datetime(year, month, 1)
    jdata_user_order_data_1 = jdata_user_order_data[['user_id', 'o_date']][(jdata_user_order_data.cate == cate) & (jdata_user_order_data.o_date < time_boundary)]
    jdata_train_df_1 = pd.merge(jdata_train_df_1, jdata_user_order_data_1, on=['user_id'], how='left')
    jdata_train_df_1 = jdata_train_df_1.sort_index(by=['user_id', 'o_date'], ascending=False)
    jdata_train_df_1 = jdata_train_df_1.drop_duplicates(['user_id'])
    jdata_train_df_1['last_buy_cate' + str(cate) + '_deltaDay'] = jdata_train_df_1['o_date'].map(lambda x: 99999999 if pd.isnull(x) else (time_boundary - x).days)
#     print(jdata_train_df_1.head(10))
#     jdata_train_df = pd.merge(jdata_train_df, jdata_train_df_1[['user_id', 'year', 'month', 'last_buy_cate101_deltaDay']], on=['user_id', 'year', 'month'], how='left')
#     print(jdata_train_df[['user_id', 'last_buy_cate101_deltaDay']][(jdata_train_df.year == 2017) & (jdata_train_df.month == 3)].head())
    return jdata_train_df_1[['user_id', 'year', 'month', 'last_buy_cate' + str(cate) + '_deltaDay']]

def getUserCateLastBuyDeltaTime(jdata_train_df, jdata_test_df, jdata_user_basic_info_data, jdata_user_order_data, cate):
    jdata_train_df_temp = pd.DataFrame(columns=['user_id', 'year', 'month', 'last_buy_cate' + str(cate) + '_deltaDay'])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLastBuyDeltaTimeMonthly(jdata_user_basic_info_data, jdata_user_order_data, 2017, 4, cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLastBuyDeltaTimeMonthly(jdata_user_basic_info_data, jdata_user_order_data, 2017, 3, cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLastBuyDeltaTimeMonthly(jdata_user_basic_info_data, jdata_user_order_data, 2017, 2, cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLastBuyDeltaTimeMonthly(jdata_user_basic_info_data, jdata_user_order_data, 2017, 1, cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLastBuyDeltaTimeMonthly(jdata_user_basic_info_data, jdata_user_order_data, 2016, 12, cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLastBuyDeltaTimeMonthly(jdata_user_basic_info_data, jdata_user_order_data, 2016, 11, cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLastBuyDeltaTimeMonthly(jdata_user_basic_info_data, jdata_user_order_data, 2016, 10, cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLastBuyDeltaTimeMonthly(jdata_user_basic_info_data, jdata_user_order_data, 2016, 9, cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLastBuyDeltaTimeMonthly(jdata_user_basic_info_data, jdata_user_order_data, 2016, 8, cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLastBuyDeltaTimeMonthly(jdata_user_basic_info_data, jdata_user_order_data, 2016, 7, cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLastBuyDeltaTimeMonthly(jdata_user_basic_info_data, jdata_user_order_data, 2016, 6, cate)])
#     jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLastBuyDeltaTimeMonthly(jdata_user_basic_info_data, jdata_user_order_data, 2016, 5, cate)])
    jdata_train_df = pd.merge(jdata_train_df, jdata_train_df_temp, on=['user_id', 'year', 'month'], how='left')
    jdata_test_df_temp = getUserCateLastBuyDeltaTimeMonthly(jdata_user_basic_info_data, jdata_user_order_data, 2017, 5, cate)
    jdata_test_df = pd.merge(jdata_test_df, jdata_test_df_temp, on=['user_id', 'year', 'month'], how='left')
    return jdata_train_df, jdata_test_df

print(len(jdata_train_df))
print(len(jdata_train_df.columns.values))
print(len(jdata_test_df))
print(len(jdata_test_df.columns.values))
jdata_train_df, jdata_test_df = getUserCateLastBuyDeltaTime(jdata_train_df, jdata_test_df, jdata_user_basic_info_data, jdata_user_order_data, 101)
jdata_train_df, jdata_test_df = getUserCateLastBuyDeltaTime(jdata_train_df, jdata_test_df, jdata_user_basic_info_data, jdata_user_order_data, 30)
jdata_train_df, jdata_test_df = getUserCateLastBuyDeltaTime(jdata_train_df, jdata_test_df, jdata_user_basic_info_data, jdata_user_order_data, 1)
jdata_train_df, jdata_test_df = getUserCateLastBuyDeltaTime(jdata_train_df, jdata_test_df, jdata_user_basic_info_data, jdata_user_order_data, 71)
jdata_train_df, jdata_test_df = getUserCateLastBuyDeltaTime(jdata_train_df, jdata_test_df, jdata_user_basic_info_data, jdata_user_order_data, 46)
jdata_train_df, jdata_test_df = getUserCateLastBuyDeltaTime(jdata_train_df, jdata_test_df, jdata_user_basic_info_data, jdata_user_order_data, 83)
print(len(jdata_train_df))
print(len(jdata_train_df.columns.values))
print(len(jdata_test_df))
print(len(jdata_test_df.columns.values))


1088164
8
98924
6


/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: FutureWarning: by argument to sort_index is deprecated, pls use .sort_values(by=...)
  # Remove the CWD from sys.path while we load stuff.


1088164
14
98924
12


In [13]:
# 统计某用户距离上次购买目标类目商品的时间，逐个月进行处理
def getUserTargetCateLastBuyDeltaTimeMonthly(jdata_user_basic_info_data, jdata_user_order_data, year, month):
    jdata_train_df_1 = jdata_user_basic_info_data.copy()
    jdata_train_df_1['month'] = month
    jdata_train_df_1['year'] = year
    time_boundary = datetime.datetime(year, month, 1)
    jdata_user_order_data_1 = jdata_user_order_data[['user_id', 'o_date']][((jdata_user_order_data.cate == 101) | (jdata_user_order_data.cate == 30)) & (jdata_user_order_data.o_date < time_boundary)]
    jdata_train_df_1 = pd.merge(jdata_train_df_1, jdata_user_order_data_1, on=['user_id'], how='left')
    jdata_train_df_1 = jdata_train_df_1.sort_index(by=['user_id', 'o_date'], ascending=False)
    jdata_train_df_1 = jdata_train_df_1.drop_duplicates(['user_id'])
    jdata_train_df_1['last_buy_targetCate_deltaDay'] = jdata_train_df_1['o_date'].map(lambda x: 99999999 if pd.isnull(x) else (time_boundary - x).days)
    return jdata_train_df_1[['user_id', 'year', 'month', 'last_buy_targetCate_deltaDay']]

def getUserTargetCateLastBuyDeltaTime(jdata_train_df, jdata_test_df, jdata_user_basic_info_data, jdata_user_order_data):
    jdata_train_df_temp = pd.DataFrame(columns=['user_id', 'year', 'month', 'last_buy_targetCate_deltaDay'])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserTargetCateLastBuyDeltaTimeMonthly(jdata_user_basic_info_data, jdata_user_order_data, 2017, 4)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserTargetCateLastBuyDeltaTimeMonthly(jdata_user_basic_info_data, jdata_user_order_data, 2017, 3)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserTargetCateLastBuyDeltaTimeMonthly(jdata_user_basic_info_data, jdata_user_order_data, 2017, 2)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserTargetCateLastBuyDeltaTimeMonthly(jdata_user_basic_info_data, jdata_user_order_data, 2017, 1)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserTargetCateLastBuyDeltaTimeMonthly(jdata_user_basic_info_data, jdata_user_order_data, 2016, 12)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserTargetCateLastBuyDeltaTimeMonthly(jdata_user_basic_info_data, jdata_user_order_data, 2016, 11)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserTargetCateLastBuyDeltaTimeMonthly(jdata_user_basic_info_data, jdata_user_order_data, 2016, 10)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserTargetCateLastBuyDeltaTimeMonthly(jdata_user_basic_info_data, jdata_user_order_data, 2016, 9)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserTargetCateLastBuyDeltaTimeMonthly(jdata_user_basic_info_data, jdata_user_order_data, 2016, 8)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserTargetCateLastBuyDeltaTimeMonthly(jdata_user_basic_info_data, jdata_user_order_data, 2016, 7)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserTargetCateLastBuyDeltaTimeMonthly(jdata_user_basic_info_data, jdata_user_order_data, 2016, 6)])
    jdata_train_df = pd.merge(jdata_train_df, jdata_train_df_temp, on=['user_id', 'year', 'month'], how='left')
    jdata_test_df_temp = getUserTargetCateLastBuyDeltaTimeMonthly(jdata_user_basic_info_data, jdata_user_order_data, 2017, 5)
    jdata_test_df = pd.merge(jdata_test_df, jdata_test_df_temp, on=['user_id', 'year', 'month'], how='left')
    return jdata_train_df, jdata_test_df

print(len(jdata_train_df))
print(len(jdata_train_df.columns.values))
print(len(jdata_test_df))
print(len(jdata_test_df.columns.values))
jdata_train_df, jdata_test_df = getUserTargetCateLastBuyDeltaTime(jdata_train_df, jdata_test_df, jdata_user_basic_info_data, jdata_user_order_data)
print(len(jdata_train_df))
print(len(jdata_train_df.columns.values))
print(len(jdata_test_df))
print(len(jdata_test_df.columns.values))


1088164
14
98924
12


/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: FutureWarning: by argument to sort_index is deprecated, pls use .sort_values(by=...)
  if __name__ == '__main__':


1088164
15
98924
13


In [14]:
# 统计某用户距离上次购买相关类目商品的时间，逐个月进行处理
def getUserRelatedCateLastBuyDeltaTimeMonthly(jdata_user_basic_info_data, jdata_user_order_data, year, month):
    jdata_train_df_1 = jdata_user_basic_info_data.copy()
    jdata_train_df_1['month'] = month
    jdata_train_df_1['year'] = year
    time_boundary = datetime.datetime(year, month, 1)
    jdata_user_order_data_1 = jdata_user_order_data[['user_id', 'o_date']][((jdata_user_order_data.cate != 101) & (jdata_user_order_data.cate != 30)) & (jdata_user_order_data.o_date < time_boundary)]
    jdata_train_df_1 = pd.merge(jdata_train_df_1, jdata_user_order_data_1, on=['user_id'], how='left')
    jdata_train_df_1 = jdata_train_df_1.sort_index(by=['user_id', 'o_date'], ascending=False)
    jdata_train_df_1 = jdata_train_df_1.drop_duplicates(['user_id'])
    jdata_train_df_1['last_buy_relatedCate_deltaDay'] = jdata_train_df_1['o_date'].map(lambda x: 99999999 if pd.isnull(x) else (time_boundary - x).days)
    return jdata_train_df_1[['user_id', 'year', 'month', 'last_buy_relatedCate_deltaDay']]

def getUserRelatedCateLastBuyDeltaTime(jdata_train_df, jdata_test_df, jdata_user_basic_info_data, jdata_user_order_data):
    jdata_train_df_temp = pd.DataFrame(columns=['user_id', 'year', 'month', 'last_buy_relatedCate_deltaDay'])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserRelatedCateLastBuyDeltaTimeMonthly(jdata_user_basic_info_data, jdata_user_order_data, 2017, 4)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserRelatedCateLastBuyDeltaTimeMonthly(jdata_user_basic_info_data, jdata_user_order_data, 2017, 3)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserRelatedCateLastBuyDeltaTimeMonthly(jdata_user_basic_info_data, jdata_user_order_data, 2017, 2)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserRelatedCateLastBuyDeltaTimeMonthly(jdata_user_basic_info_data, jdata_user_order_data, 2017, 1)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserRelatedCateLastBuyDeltaTimeMonthly(jdata_user_basic_info_data, jdata_user_order_data, 2016, 12)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserRelatedCateLastBuyDeltaTimeMonthly(jdata_user_basic_info_data, jdata_user_order_data, 2016, 11)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserRelatedCateLastBuyDeltaTimeMonthly(jdata_user_basic_info_data, jdata_user_order_data, 2016, 10)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserRelatedCateLastBuyDeltaTimeMonthly(jdata_user_basic_info_data, jdata_user_order_data, 2016, 9)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserRelatedCateLastBuyDeltaTimeMonthly(jdata_user_basic_info_data, jdata_user_order_data, 2016, 8)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserRelatedCateLastBuyDeltaTimeMonthly(jdata_user_basic_info_data, jdata_user_order_data, 2016, 7)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserRelatedCateLastBuyDeltaTimeMonthly(jdata_user_basic_info_data, jdata_user_order_data, 2016, 6)])
    jdata_train_df = pd.merge(jdata_train_df, jdata_train_df_temp, on=['user_id', 'year', 'month'], how='left')
    jdata_test_df_temp = getUserRelatedCateLastBuyDeltaTimeMonthly(jdata_user_basic_info_data, jdata_user_order_data, 2017, 5)
    jdata_test_df = pd.merge(jdata_test_df, jdata_test_df_temp, on=['user_id', 'year', 'month'], how='left')
    return jdata_train_df, jdata_test_df

print(len(jdata_train_df))
print(len(jdata_train_df.columns.values))
print(len(jdata_test_df))
print(len(jdata_test_df.columns.values))
jdata_train_df, jdata_test_df = getUserRelatedCateLastBuyDeltaTime(jdata_train_df, jdata_test_df, jdata_user_basic_info_data, jdata_user_order_data)
print(len(jdata_train_df))
print(len(jdata_train_df.columns.values))
print(len(jdata_test_df))
print(len(jdata_test_df.columns.values))


1088164
15
98924
13


/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: FutureWarning: by argument to sort_index is deprecated, pls use .sort_values(by=...)
  if __name__ == '__main__':


1088164
16
98924
14


In [15]:
print(len(jdata_user_action_data))
jdata_user_action_data = pd.merge(jdata_user_action_data, jdata_sku_basic_info_data[['sku_id', 'cate']], on=['sku_id'], how='left')
print(len(jdata_user_action_data))

# 统计某用户关注某个类目商品的次数（历史），逐个月进行处理
def getUserCateFocusNumberMonthly(jdata_user_basic_info_data, jdata_user_action_data, year, month, cate):
    jdata_train_df_1 = jdata_user_basic_info_data.copy()
    jdata_train_df_1['month'] = month
    jdata_train_df_1['year'] = year
    jdata_train_df_1['cate'] = cate
    time_boundary = datetime.datetime(year, month, 1)
    jdata_user_action_data_1 = jdata_user_action_data[['user_id', 'a_date', 'a_num']][(jdata_user_action_data.cate == cate) & (jdata_user_action_data.a_date < time_boundary) & (jdata_user_action_data.a_type == 2)]
    jdata_user_action_data_pivot_table = pd.pivot_table(jdata_user_action_data_1, index=['user_id'], values=['a_num'], aggfunc=sum)
    jdata_user_action_data_pivot_table.reset_index(inplace=True)
#     print(jdata_user_action_data_pivot_table.head(10))
    jdata_train_df_1 = pd.merge(jdata_train_df_1, jdata_user_action_data_pivot_table, on=['user_id'], how='left')
    jdata_train_df_1['last_focus_cate' + str(cate) + '_number'] = jdata_train_df_1['a_num'].map(lambda x: 0 if pd.isnull(x) else x)
    return jdata_train_df_1[['user_id', 'year', 'month', 'last_focus_cate' + str(cate) + '_number']]


def getUserCateFocusNumber(jdata_train_df, jdata_test_df, jdata_user_basic_info_data, jdata_user_action_data, cate):
    jdata_train_df_temp = pd.DataFrame(columns=['user_id', 'year', 'month', 'last_focus_cate' + str(cate) + '_number'])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateFocusNumberMonthly(jdata_user_basic_info_data, jdata_user_action_data, 2017, 4, cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateFocusNumberMonthly(jdata_user_basic_info_data, jdata_user_action_data, 2017, 3, cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateFocusNumberMonthly(jdata_user_basic_info_data, jdata_user_action_data, 2017, 2, cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateFocusNumberMonthly(jdata_user_basic_info_data, jdata_user_action_data, 2017, 1, cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateFocusNumberMonthly(jdata_user_basic_info_data, jdata_user_action_data, 2016, 12, cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateFocusNumberMonthly(jdata_user_basic_info_data, jdata_user_action_data, 2016, 11, cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateFocusNumberMonthly(jdata_user_basic_info_data, jdata_user_action_data, 2016, 10, cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateFocusNumberMonthly(jdata_user_basic_info_data, jdata_user_action_data, 2016, 9, cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateFocusNumberMonthly(jdata_user_basic_info_data, jdata_user_action_data, 2016, 8, cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateFocusNumberMonthly(jdata_user_basic_info_data, jdata_user_action_data, 2016, 7, cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateFocusNumberMonthly(jdata_user_basic_info_data, jdata_user_action_data, 2016, 6, cate)])
#     jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateFocusNumberMonthly(jdata_user_basic_info_data, jdata_user_action_data, 2016, 5, cate)])
    jdata_train_df = pd.merge(jdata_train_df, jdata_train_df_temp, on=['user_id', 'year', 'month'], how='left')
    jdata_test_df_temp = getUserCateFocusNumberMonthly(jdata_user_basic_info_data, jdata_user_action_data, 2017, 5, cate)
    jdata_test_df = pd.merge(jdata_test_df, jdata_test_df_temp, on=['user_id', 'year', 'month'], how='left')
    return jdata_train_df, jdata_test_df

print(len(jdata_train_df))
print(len(jdata_train_df.columns.values))
print(len(jdata_test_df))
print(len(jdata_test_df.columns.values))
jdata_train_df, jdata_test_df = getUserCateFocusNumber(jdata_train_df, jdata_test_df, jdata_user_basic_info_data, jdata_user_action_data, 101)
jdata_train_df, jdata_test_df = getUserCateFocusNumber(jdata_train_df, jdata_test_df, jdata_user_basic_info_data, jdata_user_action_data, 30)
jdata_train_df, jdata_test_df = getUserCateFocusNumber(jdata_train_df, jdata_test_df, jdata_user_basic_info_data, jdata_user_action_data, 1)
jdata_train_df, jdata_test_df = getUserCateFocusNumber(jdata_train_df, jdata_test_df, jdata_user_basic_info_data, jdata_user_action_data, 71)
jdata_train_df, jdata_test_df = getUserCateFocusNumber(jdata_train_df, jdata_test_df, jdata_user_basic_info_data, jdata_user_action_data, 46)
jdata_train_df, jdata_test_df = getUserCateFocusNumber(jdata_train_df, jdata_test_df, jdata_user_basic_info_data, jdata_user_action_data, 83)
print(len(jdata_train_df))
print(len(jdata_train_df.columns.values))
print(len(jdata_test_df))
print(len(jdata_test_df.columns.values))


6944141
6944141
1088164
16
98924
14
1088164
22
98924
20


In [16]:
# 统计某用户关注目标类目商品的次数（历史），逐个月进行处理
def getUserTargetCateFocusNumberMonthly(jdata_user_basic_info_data, jdata_user_action_data, year, month):
    jdata_train_df_1 = jdata_user_basic_info_data.copy()
    jdata_train_df_1['month'] = month
    jdata_train_df_1['year'] = year
    time_boundary = datetime.datetime(year, month, 1)
    jdata_user_action_data_1 = jdata_user_action_data[['user_id', 'a_date', 'a_num']][((jdata_user_action_data.cate == 101) | (jdata_user_action_data.cate == 30)) & (jdata_user_action_data.a_date < time_boundary) & (jdata_user_action_data.a_type == 2)]
    jdata_user_action_data_pivot_table = pd.pivot_table(jdata_user_action_data_1, index=['user_id'], values=['a_num'], aggfunc=sum)
    jdata_user_action_data_pivot_table.reset_index(inplace=True)
    jdata_train_df_1 = pd.merge(jdata_train_df_1, jdata_user_action_data_pivot_table, on=['user_id'], how='left')
    jdata_train_df_1['last_focus_targetCate_number'] = jdata_train_df_1['a_num'].map(lambda x: 0 if pd.isnull(x) else x)
    return jdata_train_df_1[['user_id', 'year', 'month', 'last_focus_targetCate_number']]


def getUserTargetCateFocusNumber(jdata_train_df, jdata_test_df, jdata_user_basic_info_data, jdata_user_action_data):
    jdata_train_df_temp = pd.DataFrame(columns=['user_id', 'year', 'month', 'last_focus_targetCate_number'])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserTargetCateFocusNumberMonthly(jdata_user_basic_info_data, jdata_user_action_data, 2017, 4)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserTargetCateFocusNumberMonthly(jdata_user_basic_info_data, jdata_user_action_data, 2017, 3)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserTargetCateFocusNumberMonthly(jdata_user_basic_info_data, jdata_user_action_data, 2017, 2)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserTargetCateFocusNumberMonthly(jdata_user_basic_info_data, jdata_user_action_data, 2017, 1)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserTargetCateFocusNumberMonthly(jdata_user_basic_info_data, jdata_user_action_data, 2016, 12)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserTargetCateFocusNumberMonthly(jdata_user_basic_info_data, jdata_user_action_data, 2016, 11)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserTargetCateFocusNumberMonthly(jdata_user_basic_info_data, jdata_user_action_data, 2016, 10)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserTargetCateFocusNumberMonthly(jdata_user_basic_info_data, jdata_user_action_data, 2016, 9)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserTargetCateFocusNumberMonthly(jdata_user_basic_info_data, jdata_user_action_data, 2016, 8)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserTargetCateFocusNumberMonthly(jdata_user_basic_info_data, jdata_user_action_data, 2016, 7)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserTargetCateFocusNumberMonthly(jdata_user_basic_info_data, jdata_user_action_data, 2016, 6)])
    jdata_train_df = pd.merge(jdata_train_df, jdata_train_df_temp, on=['user_id', 'year', 'month'], how='left')
    jdata_test_df_temp = getUserTargetCateFocusNumberMonthly(jdata_user_basic_info_data, jdata_user_action_data, 2017, 5)
    jdata_test_df = pd.merge(jdata_test_df, jdata_test_df_temp, on=['user_id', 'year', 'month'], how='left')
    return jdata_train_df, jdata_test_df

print(len(jdata_train_df))
print(len(jdata_train_df.columns.values))
print(len(jdata_test_df))
print(len(jdata_test_df.columns.values))
jdata_train_df, jdata_test_df = getUserTargetCateFocusNumber(jdata_train_df, jdata_test_df, jdata_user_basic_info_data, jdata_user_action_data)
print(len(jdata_train_df))
print(len(jdata_train_df.columns.values))
print(len(jdata_test_df))
print(len(jdata_test_df.columns.values))


1088164
22
98924
20
1088164
23
98924
21


In [17]:
# 统计某用户关注相关类目商品的次数（历史），逐个月进行处理
def getUserRelatedCateFocusNumberMonthly(jdata_user_basic_info_data, jdata_user_action_data, year, month):
    jdata_train_df_1 = jdata_user_basic_info_data.copy()
    jdata_train_df_1['month'] = month
    jdata_train_df_1['year'] = year
    time_boundary = datetime.datetime(year, month, 1)
    jdata_user_action_data_1 = jdata_user_action_data[['user_id', 'a_date', 'a_num']][((jdata_user_action_data.cate != 101) & (jdata_user_action_data.cate != 30)) & (jdata_user_action_data.a_date < time_boundary) & (jdata_user_action_data.a_type == 2)]
    jdata_user_action_data_pivot_table = pd.pivot_table(jdata_user_action_data_1, index=['user_id'], values=['a_num'], aggfunc=sum)
    jdata_user_action_data_pivot_table.reset_index(inplace=True)
    jdata_train_df_1 = pd.merge(jdata_train_df_1, jdata_user_action_data_pivot_table, on=['user_id'], how='left')
    jdata_train_df_1['last_focus_relatedCate_number'] = jdata_train_df_1['a_num'].map(lambda x: 0 if pd.isnull(x) else x)
    return jdata_train_df_1[['user_id', 'year', 'month', 'last_focus_relatedCate_number']]


def getUserRelatedCateFocusNumber(jdata_train_df, jdata_test_df, jdata_user_basic_info_data, jdata_user_action_data):
    jdata_train_df_temp = pd.DataFrame(columns=['user_id', 'year', 'month', 'last_focus_relatedCate_number'])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserRelatedCateFocusNumberMonthly(jdata_user_basic_info_data, jdata_user_action_data, 2017, 4)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserRelatedCateFocusNumberMonthly(jdata_user_basic_info_data, jdata_user_action_data, 2017, 3)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserRelatedCateFocusNumberMonthly(jdata_user_basic_info_data, jdata_user_action_data, 2017, 2)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserRelatedCateFocusNumberMonthly(jdata_user_basic_info_data, jdata_user_action_data, 2017, 1)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserRelatedCateFocusNumberMonthly(jdata_user_basic_info_data, jdata_user_action_data, 2016, 12)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserRelatedCateFocusNumberMonthly(jdata_user_basic_info_data, jdata_user_action_data, 2016, 11)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserRelatedCateFocusNumberMonthly(jdata_user_basic_info_data, jdata_user_action_data, 2016, 10)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserRelatedCateFocusNumberMonthly(jdata_user_basic_info_data, jdata_user_action_data, 2016, 9)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserRelatedCateFocusNumberMonthly(jdata_user_basic_info_data, jdata_user_action_data, 2016, 8)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserRelatedCateFocusNumberMonthly(jdata_user_basic_info_data, jdata_user_action_data, 2016, 7)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserRelatedCateFocusNumberMonthly(jdata_user_basic_info_data, jdata_user_action_data, 2016, 6)])
    jdata_train_df = pd.merge(jdata_train_df, jdata_train_df_temp, on=['user_id', 'year', 'month'], how='left')
    jdata_test_df_temp = getUserRelatedCateFocusNumberMonthly(jdata_user_basic_info_data, jdata_user_action_data, 2017, 5)
    jdata_test_df = pd.merge(jdata_test_df, jdata_test_df_temp, on=['user_id', 'year', 'month'], how='left')
    return jdata_train_df, jdata_test_df

print(len(jdata_train_df))
print(len(jdata_train_df.columns.values))
print(len(jdata_test_df))
print(len(jdata_test_df.columns.values))
jdata_train_df, jdata_test_df = getUserRelatedCateFocusNumber(jdata_train_df, jdata_test_df, jdata_user_basic_info_data, jdata_user_action_data)
print(len(jdata_train_df))
print(len(jdata_train_df.columns.values))
print(len(jdata_test_df))
print(len(jdata_test_df.columns.values))


1088164
23
98924
21
1088164
24
98924
22


In [18]:
# 统计某用户距离上次点击相同类目商品的时间，逐个月进行处理
def getUserCateLastClickDeltaTimeMonthly(jdata_user_basic_info_data, jdata_user_action_data, year, month, cate):
    jdata_train_df_1 = jdata_user_basic_info_data.copy()
    jdata_train_df_1['month'] = month
    jdata_train_df_1['year'] = year
    jdata_train_df_1['cate'] = cate
    time_boundary = datetime.datetime(year, month, 1)
    jdata_user_action_data_1 = jdata_user_action_data[['user_id', 'a_date']][(jdata_user_action_data.cate == cate) & (jdata_user_action_data.a_date < time_boundary)]
    jdata_train_df_1 = pd.merge(jdata_train_df_1, jdata_user_action_data_1, on=['user_id'], how='left')
    jdata_train_df_1 = jdata_train_df_1.sort_index(by=['user_id', 'a_date'], ascending=False)
#     print(jdata_train_df_1.head(10))
    jdata_train_df_1 = jdata_train_df_1.drop_duplicates(['user_id'])
    jdata_train_df_1['last_click_cate' + str(cate) + '_deltaDay'] = jdata_train_df_1['a_date'].map(lambda x: 99999999 if pd.isnull(x) else (time_boundary - x).days)
    return jdata_train_df_1[['user_id', 'year', 'month', 'last_click_cate' + str(cate) + '_deltaDay']]

def getUserCateLastClickDeltaTime(jdata_train_df, jdata_test_df, jdata_user_basic_info_data, jdata_user_action_data, cate):
    jdata_train_df_temp = pd.DataFrame(columns=['user_id', 'year', 'month', 'last_click_cate' + str(cate) + '_deltaDay'])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLastClickDeltaTimeMonthly(jdata_user_basic_info_data, jdata_user_action_data, 2017, 4, cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLastClickDeltaTimeMonthly(jdata_user_basic_info_data, jdata_user_action_data, 2017, 3, cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLastClickDeltaTimeMonthly(jdata_user_basic_info_data, jdata_user_action_data, 2017, 2, cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLastClickDeltaTimeMonthly(jdata_user_basic_info_data, jdata_user_action_data, 2017, 1, cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLastClickDeltaTimeMonthly(jdata_user_basic_info_data, jdata_user_action_data, 2016, 12, cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLastClickDeltaTimeMonthly(jdata_user_basic_info_data, jdata_user_action_data, 2016, 11, cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLastClickDeltaTimeMonthly(jdata_user_basic_info_data, jdata_user_action_data, 2016, 10, cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLastClickDeltaTimeMonthly(jdata_user_basic_info_data, jdata_user_action_data, 2016, 9, cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLastClickDeltaTimeMonthly(jdata_user_basic_info_data, jdata_user_action_data, 2016, 8, cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLastClickDeltaTimeMonthly(jdata_user_basic_info_data, jdata_user_action_data, 2016, 7, cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLastClickDeltaTimeMonthly(jdata_user_basic_info_data, jdata_user_action_data, 2016, 6, cate)])
#     jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLastClickDeltaTimeMonthly(jdata_user_basic_info_data, jdata_user_action_data, 2016, 5, cate)])
    jdata_train_df = pd.merge(jdata_train_df, jdata_train_df_temp, on=['user_id', 'year', 'month'], how='left')
    jdata_test_df_temp = getUserCateLastClickDeltaTimeMonthly(jdata_user_basic_info_data, jdata_user_action_data, 2017, 5, cate)
    jdata_test_df = pd.merge(jdata_test_df, jdata_test_df_temp, on=['user_id', 'year', 'month'], how='left')
    return jdata_train_df, jdata_test_df

print(len(jdata_train_df))
print(len(jdata_train_df.columns.values))
print(len(jdata_test_df))
print(len(jdata_test_df.columns.values))
jdata_train_df, jdata_test_df = getUserCateLastClickDeltaTime(jdata_train_df, jdata_test_df, jdata_user_basic_info_data, jdata_user_action_data, 101)
jdata_train_df, jdata_test_df = getUserCateLastClickDeltaTime(jdata_train_df, jdata_test_df, jdata_user_basic_info_data, jdata_user_action_data, 30)
jdata_train_df, jdata_test_df = getUserCateLastClickDeltaTime(jdata_train_df, jdata_test_df, jdata_user_basic_info_data, jdata_user_action_data, 1)
jdata_train_df, jdata_test_df = getUserCateLastClickDeltaTime(jdata_train_df, jdata_test_df, jdata_user_basic_info_data, jdata_user_action_data, 71)
jdata_train_df, jdata_test_df = getUserCateLastClickDeltaTime(jdata_train_df, jdata_test_df, jdata_user_basic_info_data, jdata_user_action_data, 46)
jdata_train_df, jdata_test_df = getUserCateLastClickDeltaTime(jdata_train_df, jdata_test_df, jdata_user_basic_info_data, jdata_user_action_data, 83)
print(len(jdata_train_df))
print(len(jdata_train_df.columns.values))
print(len(jdata_test_df))
print(len(jdata_test_df.columns.values))


1088164
24
98924
22


/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: FutureWarning: by argument to sort_index is deprecated, pls use .sort_values(by=...)
  # Remove the CWD from sys.path while we load stuff.


1088164
30
98924
28


In [19]:
# 统计某用户距离上次点击目标类目商品的时间，逐个月进行处理
def getUserTargetCateLastClickDeltaTimeMonthly(jdata_user_basic_info_data, jdata_user_action_data, year, month):
    jdata_train_df_1 = jdata_user_basic_info_data.copy()
    jdata_train_df_1['month'] = month
    jdata_train_df_1['year'] = year
    time_boundary = datetime.datetime(year, month, 1)
    jdata_user_action_data_1 = jdata_user_action_data[['user_id', 'a_date']][((jdata_user_action_data.cate == 101) | (jdata_user_action_data.cate == 30)) & (jdata_user_action_data.a_date < time_boundary)]
    jdata_train_df_1 = pd.merge(jdata_train_df_1, jdata_user_action_data_1, on=['user_id'], how='left')
    jdata_train_df_1 = jdata_train_df_1.sort_index(by=['user_id', 'a_date'], ascending=False)
    jdata_train_df_1 = jdata_train_df_1.drop_duplicates(['user_id'])
    jdata_train_df_1['last_click_TargetCate_deltaDay'] = jdata_train_df_1['a_date'].map(lambda x: 99999999 if pd.isnull(x) else (time_boundary - x).days)
    return jdata_train_df_1[['user_id', 'year', 'month', 'last_click_TargetCate_deltaDay']]

def getUserTargetCateLastClickDeltaTime(jdata_train_df, jdata_test_df, jdata_user_basic_info_data, jdata_user_action_data):
    jdata_train_df_temp = pd.DataFrame(columns=['user_id', 'year', 'month', 'last_click_TargetCate_deltaDay'])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserTargetCateLastClickDeltaTimeMonthly(jdata_user_basic_info_data, jdata_user_action_data, 2017, 4)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserTargetCateLastClickDeltaTimeMonthly(jdata_user_basic_info_data, jdata_user_action_data, 2017, 3)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserTargetCateLastClickDeltaTimeMonthly(jdata_user_basic_info_data, jdata_user_action_data, 2017, 2)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserTargetCateLastClickDeltaTimeMonthly(jdata_user_basic_info_data, jdata_user_action_data, 2017, 1)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserTargetCateLastClickDeltaTimeMonthly(jdata_user_basic_info_data, jdata_user_action_data, 2016, 12)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserTargetCateLastClickDeltaTimeMonthly(jdata_user_basic_info_data, jdata_user_action_data, 2016, 11)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserTargetCateLastClickDeltaTimeMonthly(jdata_user_basic_info_data, jdata_user_action_data, 2016, 10)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserTargetCateLastClickDeltaTimeMonthly(jdata_user_basic_info_data, jdata_user_action_data, 2016, 9)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserTargetCateLastClickDeltaTimeMonthly(jdata_user_basic_info_data, jdata_user_action_data, 2016, 8)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserTargetCateLastClickDeltaTimeMonthly(jdata_user_basic_info_data, jdata_user_action_data, 2016, 7)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserTargetCateLastClickDeltaTimeMonthly(jdata_user_basic_info_data, jdata_user_action_data, 2016, 6)])
    jdata_train_df = pd.merge(jdata_train_df, jdata_train_df_temp, on=['user_id', 'year', 'month'], how='left')
    jdata_test_df_temp = getUserTargetCateLastClickDeltaTimeMonthly(jdata_user_basic_info_data, jdata_user_action_data, 2017, 5)
    jdata_test_df = pd.merge(jdata_test_df, jdata_test_df_temp, on=['user_id', 'year', 'month'], how='left')
    return jdata_train_df, jdata_test_df

print(len(jdata_train_df))
print(len(jdata_train_df.columns.values))
print(len(jdata_test_df))
print(len(jdata_test_df.columns.values))
jdata_train_df, jdata_test_df = getUserTargetCateLastClickDeltaTime(jdata_train_df, jdata_test_df, jdata_user_basic_info_data, jdata_user_action_data)
print(len(jdata_train_df))
print(len(jdata_train_df.columns.values))
print(len(jdata_test_df))
print(len(jdata_test_df.columns.values))


1088164
30
98924
28


/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: FutureWarning: by argument to sort_index is deprecated, pls use .sort_values(by=...)
  if __name__ == '__main__':


1088164
31
98924
29


In [20]:
# 统计某用户距离上次点击相关类目商品的时间，逐个月进行处理
def getUserRelatedCateLastClickDeltaTimeMonthly(jdata_user_basic_info_data, jdata_user_action_data, year, month):
    jdata_train_df_1 = jdata_user_basic_info_data.copy()
    jdata_train_df_1['month'] = month
    jdata_train_df_1['year'] = year
    time_boundary = datetime.datetime(year, month, 1)
    jdata_user_action_data_1 = jdata_user_action_data[['user_id', 'a_date']][((jdata_user_action_data.cate != 101) & (jdata_user_action_data.cate != 30)) & (jdata_user_action_data.a_date < time_boundary)]
    jdata_train_df_1 = pd.merge(jdata_train_df_1, jdata_user_action_data_1, on=['user_id'], how='left')
    jdata_train_df_1 = jdata_train_df_1.sort_index(by=['user_id', 'a_date'], ascending=False)
    jdata_train_df_1 = jdata_train_df_1.drop_duplicates(['user_id'])
    jdata_train_df_1['last_click_relatedCate_deltaDay'] = jdata_train_df_1['a_date'].map(lambda x: 99999999 if pd.isnull(x) else (time_boundary - x).days)
    return jdata_train_df_1[['user_id', 'year', 'month', 'last_click_relatedCate_deltaDay']]

def getUserRelatedCateLastClickDeltaTime(jdata_train_df, jdata_test_df, jdata_user_basic_info_data, jdata_user_action_data):
    jdata_train_df_temp = pd.DataFrame(columns=['user_id', 'year', 'month', 'last_click_relatedCate_deltaDay'])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserRelatedCateLastClickDeltaTimeMonthly(jdata_user_basic_info_data, jdata_user_action_data, 2017, 4)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserRelatedCateLastClickDeltaTimeMonthly(jdata_user_basic_info_data, jdata_user_action_data, 2017, 3)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserRelatedCateLastClickDeltaTimeMonthly(jdata_user_basic_info_data, jdata_user_action_data, 2017, 2)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserRelatedCateLastClickDeltaTimeMonthly(jdata_user_basic_info_data, jdata_user_action_data, 2017, 1)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserRelatedCateLastClickDeltaTimeMonthly(jdata_user_basic_info_data, jdata_user_action_data, 2016, 12)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserRelatedCateLastClickDeltaTimeMonthly(jdata_user_basic_info_data, jdata_user_action_data, 2016, 11)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserRelatedCateLastClickDeltaTimeMonthly(jdata_user_basic_info_data, jdata_user_action_data, 2016, 10)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserRelatedCateLastClickDeltaTimeMonthly(jdata_user_basic_info_data, jdata_user_action_data, 2016, 9)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserRelatedCateLastClickDeltaTimeMonthly(jdata_user_basic_info_data, jdata_user_action_data, 2016, 8)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserRelatedCateLastClickDeltaTimeMonthly(jdata_user_basic_info_data, jdata_user_action_data, 2016, 7)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserRelatedCateLastClickDeltaTimeMonthly(jdata_user_basic_info_data, jdata_user_action_data, 2016, 6)])
    jdata_train_df = pd.merge(jdata_train_df, jdata_train_df_temp, on=['user_id', 'year', 'month'], how='left')
    jdata_test_df_temp = getUserRelatedCateLastClickDeltaTimeMonthly(jdata_user_basic_info_data, jdata_user_action_data, 2017, 5)
    jdata_test_df = pd.merge(jdata_test_df, jdata_test_df_temp, on=['user_id', 'year', 'month'], how='left')
    return jdata_train_df, jdata_test_df

print(len(jdata_train_df))
print(len(jdata_train_df.columns.values))
print(len(jdata_test_df))
print(len(jdata_test_df.columns.values))
jdata_train_df, jdata_test_df = getUserRelatedCateLastClickDeltaTime(jdata_train_df, jdata_test_df, jdata_user_basic_info_data, jdata_user_action_data)
print(len(jdata_train_df))
print(len(jdata_train_df.columns.values))
print(len(jdata_test_df))
print(len(jdata_test_df.columns.values))


1088164
31
98924
29


/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: FutureWarning: by argument to sort_index is deprecated, pls use .sort_values(by=...)
  if __name__ == '__main__':


1088164
32
98924
30


In [21]:
# 统计某用户上次购买相同类目商品的数量，逐个月进行处理
def getUserCateLastBuyNumberMonthly(jdata_user_basic_info_data, jdata_user_order_data, year, month, cate):
    jdata_train_df_1 = jdata_user_basic_info_data.copy()
    jdata_train_df_1['month'] = month
    jdata_train_df_1['year'] = year
    jdata_train_df_1['cate'] = cate
    time_boundary = datetime.datetime(year, month, 1)
    jdata_user_order_data_1 = jdata_user_order_data[['user_id', 'o_date', 'o_sku_num']][(jdata_user_order_data.cate == cate) & (jdata_user_order_data.o_date < time_boundary)]
    jdata_user_order_data_1_pivot_table = pd.pivot_table(jdata_user_order_data_1, index=['user_id', 'o_date'], values=['o_sku_num'], aggfunc=sum)
    jdata_user_order_data_1_pivot_table.reset_index(inplace=True)
    jdata_train_df_1 = pd.merge(jdata_train_df_1, jdata_user_order_data_1_pivot_table, on=['user_id'], how='left')
    jdata_train_df_1 = jdata_train_df_1.sort_index(by=['user_id', 'o_date'], ascending=False)
#     print(jdata_train_df_1.head(10))
    jdata_train_df_1 = jdata_train_df_1.drop_duplicates(['user_id'])
    jdata_train_df_1['last_buy_cate' + str(cate) + '_number'] = jdata_train_df_1['o_sku_num'].map(lambda x: 0 if pd.isnull(x) else x)
    return jdata_train_df_1[['user_id', 'year', 'month', 'last_buy_cate' + str(cate) + '_number']]

def getUserCateLastBuyNumber(jdata_train_df, jdata_test_df, jdata_user_basic_info_data, jdata_user_order_data, cate):
    jdata_train_df_temp = pd.DataFrame(columns=['user_id', 'year', 'month', 'last_buy_cate' + str(cate) + '_number'])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLastBuyNumberMonthly(jdata_user_basic_info_data, jdata_user_order_data, 2017, 4, cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLastBuyNumberMonthly(jdata_user_basic_info_data, jdata_user_order_data, 2017, 3, cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLastBuyNumberMonthly(jdata_user_basic_info_data, jdata_user_order_data, 2017, 2, cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLastBuyNumberMonthly(jdata_user_basic_info_data, jdata_user_order_data, 2017, 1, cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLastBuyNumberMonthly(jdata_user_basic_info_data, jdata_user_order_data, 2016, 12, cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLastBuyNumberMonthly(jdata_user_basic_info_data, jdata_user_order_data, 2016, 11, cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLastBuyNumberMonthly(jdata_user_basic_info_data, jdata_user_order_data, 2016, 10, cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLastBuyNumberMonthly(jdata_user_basic_info_data, jdata_user_order_data, 2016, 9, cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLastBuyNumberMonthly(jdata_user_basic_info_data, jdata_user_order_data, 2016, 8, cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLastBuyNumberMonthly(jdata_user_basic_info_data, jdata_user_order_data, 2016, 7, cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLastBuyNumberMonthly(jdata_user_basic_info_data, jdata_user_order_data, 2016, 6, cate)])
#     jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLastBuyNumberMonthly(jdata_user_basic_info_data, jdata_user_order_data, 2016, 5, cate)])
    jdata_train_df = pd.merge(jdata_train_df, jdata_train_df_temp, on=['user_id', 'year', 'month'], how='left')
    jdata_test_df_temp = getUserCateLastBuyNumberMonthly(jdata_user_basic_info_data, jdata_user_order_data, 2017, 5, cate)
    jdata_test_df = pd.merge(jdata_test_df, jdata_test_df_temp, on=['user_id', 'year', 'month'], how='left')
    return jdata_train_df, jdata_test_df

print(len(jdata_train_df))
print(len(jdata_train_df.columns.values))
print(len(jdata_test_df))
print(len(jdata_test_df.columns.values))
jdata_train_df, jdata_test_df = getUserCateLastBuyNumber(jdata_train_df, jdata_test_df, jdata_user_basic_info_data, jdata_user_order_data, 101)
jdata_train_df, jdata_test_df = getUserCateLastBuyNumber(jdata_train_df, jdata_test_df, jdata_user_basic_info_data, jdata_user_order_data, 30)
jdata_train_df, jdata_test_df = getUserCateLastBuyNumber(jdata_train_df, jdata_test_df, jdata_user_basic_info_data, jdata_user_order_data, 1)
jdata_train_df, jdata_test_df = getUserCateLastBuyNumber(jdata_train_df, jdata_test_df, jdata_user_basic_info_data, jdata_user_order_data, 71)
jdata_train_df, jdata_test_df = getUserCateLastBuyNumber(jdata_train_df, jdata_test_df, jdata_user_basic_info_data, jdata_user_order_data, 46)
jdata_train_df, jdata_test_df = getUserCateLastBuyNumber(jdata_train_df, jdata_test_df, jdata_user_basic_info_data, jdata_user_order_data, 83)
print(len(jdata_train_df))
print(len(jdata_train_df.columns.values))
print(len(jdata_test_df))
print(len(jdata_test_df.columns.values))


1088164
32
98924
30


/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: FutureWarning: by argument to sort_index is deprecated, pls use .sort_values(by=...)
  if sys.path[0] == '':


1088164
38
98924
36


In [22]:
# 统计某用户上次购买目标类目商品的数量，逐个月进行处理
def getUserTargetCateLastBuyNumberMonthly(jdata_user_basic_info_data, jdata_user_order_data, year, month):
    jdata_train_df_1 = jdata_user_basic_info_data.copy()
    jdata_train_df_1['month'] = month
    jdata_train_df_1['year'] = year
    time_boundary = datetime.datetime(year, month, 1)
    jdata_user_order_data_1 = jdata_user_order_data[['user_id', 'o_date', 'o_sku_num']][((jdata_user_order_data.cate == 30) | ((jdata_user_order_data.cate == 101))) & (jdata_user_order_data.o_date < time_boundary)]
    jdata_user_order_data_1_pivot_table = pd.pivot_table(jdata_user_order_data_1, index=['user_id', 'o_date'], values=['o_sku_num'], aggfunc=sum)
    jdata_user_order_data_1_pivot_table.reset_index(inplace=True)
    jdata_train_df_1 = pd.merge(jdata_train_df_1, jdata_user_order_data_1_pivot_table, on=['user_id'], how='left')
    jdata_train_df_1 = jdata_train_df_1.sort_index(by=['user_id', 'o_date'], ascending=False)
    jdata_train_df_1 = jdata_train_df_1.drop_duplicates(['user_id'])
    jdata_train_df_1['last_buy_targetCate_number'] = jdata_train_df_1['o_sku_num'].map(lambda x: 0 if pd.isnull(x) else x)
    return jdata_train_df_1[['user_id', 'year', 'month', 'last_buy_targetCate_number']]

def getUserTargetCateLastBuyNumber(jdata_train_df, jdata_test_df, jdata_user_basic_info_data, jdata_user_order_data):
    jdata_train_df_temp = pd.DataFrame(columns=['user_id', 'year', 'month', 'last_buy_targetCate_number'])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserTargetCateLastBuyNumberMonthly(jdata_user_basic_info_data, jdata_user_order_data, 2017, 4)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserTargetCateLastBuyNumberMonthly(jdata_user_basic_info_data, jdata_user_order_data, 2017, 3)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserTargetCateLastBuyNumberMonthly(jdata_user_basic_info_data, jdata_user_order_data, 2017, 2)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserTargetCateLastBuyNumberMonthly(jdata_user_basic_info_data, jdata_user_order_data, 2017, 1)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserTargetCateLastBuyNumberMonthly(jdata_user_basic_info_data, jdata_user_order_data, 2016, 12)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserTargetCateLastBuyNumberMonthly(jdata_user_basic_info_data, jdata_user_order_data, 2016, 11)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserTargetCateLastBuyNumberMonthly(jdata_user_basic_info_data, jdata_user_order_data, 2016, 10)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserTargetCateLastBuyNumberMonthly(jdata_user_basic_info_data, jdata_user_order_data, 2016, 9)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserTargetCateLastBuyNumberMonthly(jdata_user_basic_info_data, jdata_user_order_data, 2016, 8)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserTargetCateLastBuyNumberMonthly(jdata_user_basic_info_data, jdata_user_order_data, 2016, 7)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserTargetCateLastBuyNumberMonthly(jdata_user_basic_info_data, jdata_user_order_data, 2016, 6)])
    jdata_train_df = pd.merge(jdata_train_df, jdata_train_df_temp, on=['user_id', 'year', 'month'], how='left')
    jdata_test_df_temp = getUserTargetCateLastBuyNumberMonthly(jdata_user_basic_info_data, jdata_user_order_data, 2017, 5)
    jdata_test_df = pd.merge(jdata_test_df, jdata_test_df_temp, on=['user_id', 'year', 'month'], how='left')
    return jdata_train_df, jdata_test_df

print(len(jdata_train_df))
print(len(jdata_train_df.columns.values))
print(len(jdata_test_df))
print(len(jdata_test_df.columns.values))
jdata_train_df, jdata_test_df = getUserTargetCateLastBuyNumber(jdata_train_df, jdata_test_df, jdata_user_basic_info_data, jdata_user_order_data)
print(len(jdata_train_df))
print(len(jdata_train_df.columns.values))
print(len(jdata_test_df))
print(len(jdata_test_df.columns.values))


1088164
38
98924
36


/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: FutureWarning: by argument to sort_index is deprecated, pls use .sort_values(by=...)
  # This is added back by InteractiveShellApp.init_path()


1088164
39
98924
37


In [23]:
# 统计某用户上次购买相关类目商品的数量，逐个月进行处理
def getUserRelatedCateLastBuyNumberMonthly(jdata_user_basic_info_data, jdata_user_order_data, year, month):
    jdata_train_df_1 = jdata_user_basic_info_data.copy()
    jdata_train_df_1['month'] = month
    jdata_train_df_1['year'] = year
    time_boundary = datetime.datetime(year, month, 1)
    jdata_user_order_data_1 = jdata_user_order_data[['user_id', 'o_date', 'o_sku_num']][((jdata_user_order_data.cate != 30) & ((jdata_user_order_data.cate != 101))) & (jdata_user_order_data.o_date < time_boundary)]
    jdata_user_order_data_1_pivot_table = pd.pivot_table(jdata_user_order_data_1, index=['user_id', 'o_date'], values=['o_sku_num'], aggfunc=sum)
    jdata_user_order_data_1_pivot_table.reset_index(inplace=True)
    jdata_train_df_1 = pd.merge(jdata_train_df_1, jdata_user_order_data_1_pivot_table, on=['user_id'], how='left')
    jdata_train_df_1 = jdata_train_df_1.sort_index(by=['user_id', 'o_date'], ascending=False)
    jdata_train_df_1 = jdata_train_df_1.drop_duplicates(['user_id'])
    jdata_train_df_1['last_buy_relatedCate_number'] = jdata_train_df_1['o_sku_num'].map(lambda x: 0 if pd.isnull(x) else x)
    return jdata_train_df_1[['user_id', 'year', 'month', 'last_buy_relatedCate_number']]

def getUserRelatedCateLastBuyNumber(jdata_train_df, jdata_test_df, jdata_user_basic_info_data, jdata_user_order_data):
    jdata_train_df_temp = pd.DataFrame(columns=['user_id', 'year', 'month', 'last_buy_relatedCate_number'])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserRelatedCateLastBuyNumberMonthly(jdata_user_basic_info_data, jdata_user_order_data, 2017, 4)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserRelatedCateLastBuyNumberMonthly(jdata_user_basic_info_data, jdata_user_order_data, 2017, 3)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserRelatedCateLastBuyNumberMonthly(jdata_user_basic_info_data, jdata_user_order_data, 2017, 2)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserRelatedCateLastBuyNumberMonthly(jdata_user_basic_info_data, jdata_user_order_data, 2017, 1)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserRelatedCateLastBuyNumberMonthly(jdata_user_basic_info_data, jdata_user_order_data, 2016, 12)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserRelatedCateLastBuyNumberMonthly(jdata_user_basic_info_data, jdata_user_order_data, 2016, 11)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserRelatedCateLastBuyNumberMonthly(jdata_user_basic_info_data, jdata_user_order_data, 2016, 10)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserRelatedCateLastBuyNumberMonthly(jdata_user_basic_info_data, jdata_user_order_data, 2016, 9)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserRelatedCateLastBuyNumberMonthly(jdata_user_basic_info_data, jdata_user_order_data, 2016, 8)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserRelatedCateLastBuyNumberMonthly(jdata_user_basic_info_data, jdata_user_order_data, 2016, 7)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserRelatedCateLastBuyNumberMonthly(jdata_user_basic_info_data, jdata_user_order_data, 2016, 6)])
    jdata_train_df = pd.merge(jdata_train_df, jdata_train_df_temp, on=['user_id', 'year', 'month'], how='left')
    jdata_test_df_temp = getUserRelatedCateLastBuyNumberMonthly(jdata_user_basic_info_data, jdata_user_order_data, 2017, 5)
    jdata_test_df = pd.merge(jdata_test_df, jdata_test_df_temp, on=['user_id', 'year', 'month'], how='left')
    return jdata_train_df, jdata_test_df

print(len(jdata_train_df))
print(len(jdata_train_df.columns.values))
print(len(jdata_test_df))
print(len(jdata_test_df.columns.values))
jdata_train_df, jdata_test_df = getUserRelatedCateLastBuyNumber(jdata_train_df, jdata_test_df, jdata_user_basic_info_data, jdata_user_order_data)
print(len(jdata_train_df))
print(len(jdata_train_df.columns.values))
print(len(jdata_test_df))
print(len(jdata_test_df.columns.values))


1088164
39
98924
37


/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: FutureWarning: by argument to sort_index is deprecated, pls use .sort_values(by=...)
  # This is added back by InteractiveShellApp.init_path()


1088164
40
98924
38


In [24]:
# 统计某用户月平均下单目标类目商品的个数
def getUserTargetCateMonthlyBuyNumber(jdata_train_df, jdata_test_df, jdata_user_order_data):
    jdata_user_order_data_1 = jdata_user_order_data[['user_id', 'o_sku_num']][(jdata_user_order_data.cate == 101) | (jdata_user_order_data.cate == 30)]
    jdata_user_order_data_pivot_table = pd.pivot_table(jdata_user_order_data_1, index=['user_id'], values=['o_sku_num'], aggfunc=sum)
    jdata_user_order_data_pivot_table.rename(columns={'o_sku_num':'user_targetCate_monthlyNumber'}, inplace=True)
    jdata_user_order_data_pivot_table.reset_index(inplace=True)
    jdata_user_order_data_pivot_table['user_targetCate_monthlyNumber'] = jdata_user_order_data_pivot_table['user_targetCate_monthlyNumber'] / 12
    jdata_train_df = pd.merge(jdata_train_df, jdata_user_order_data_pivot_table[['user_id', 'user_targetCate_monthlyNumber']], on=['user_id'], how='left')
    jdata_test_df = pd.merge(jdata_test_df, jdata_user_order_data_pivot_table[['user_id', 'user_targetCate_monthlyNumber']], on=['user_id'], how='left')
    jdata_train_df['user_targetCate_monthlyNumber'] = jdata_train_df['user_targetCate_monthlyNumber'].fillna(0)
    jdata_test_df['user_targetCate_monthlyNumber'] = jdata_test_df['user_targetCate_monthlyNumber'].fillna(0)
    return jdata_train_df, jdata_test_df

print(len(jdata_train_df))
print(len(jdata_train_df.columns.values))
print(len(jdata_test_df))
print(len(jdata_test_df.columns.values))
jdata_train_df, jdata_test_df = getUserTargetCateMonthlyBuyNumber(jdata_train_df, jdata_test_df, jdata_user_order_data)
print(len(jdata_train_df))
print(len(jdata_train_df.columns.values))
print(len(jdata_test_df))
print(len(jdata_test_df.columns.values))


1088164
40
98924
38
1088164
41
98924
39


In [25]:
# 统计某用户月平均下单某个类目商品的个数
def getUserCateMonthlyBuyNumber(jdata_train_df, jdata_test_df, jdata_user_order_data, cate):
    jdata_user_order_data_1 = jdata_user_order_data[['user_id', 'o_sku_num']][jdata_user_order_data.cate == cate]
    jdata_user_order_data_pivot_table = pd.pivot_table(jdata_user_order_data_1, index=['user_id'], values=['o_sku_num'], aggfunc=sum)
    jdata_user_order_data_pivot_table.rename(columns={'o_sku_num':'user_' + str(cate) + 'Cate_monthlyNumber'}, inplace=True)
    jdata_user_order_data_pivot_table.reset_index(inplace=True)
    jdata_user_order_data_pivot_table['user_' + str(cate) + 'Cate_monthlyNumber'] = jdata_user_order_data_pivot_table['user_' + str(cate) + 'Cate_monthlyNumber'] / 12
    jdata_train_df = pd.merge(jdata_train_df, jdata_user_order_data_pivot_table[['user_id', 'user_' + str(cate) + 'Cate_monthlyNumber']], on=['user_id'], how='left')
    jdata_test_df = pd.merge(jdata_test_df, jdata_user_order_data_pivot_table[['user_id', 'user_' + str(cate) + 'Cate_monthlyNumber']], on=['user_id'], how='left')
    jdata_train_df['user_' + str(cate) + 'Cate_monthlyNumber'] = jdata_train_df['user_' + str(cate) + 'Cate_monthlyNumber'].fillna(0)
    jdata_test_df['user_' + str(cate) + 'Cate_monthlyNumber'] = jdata_test_df['user_' + str(cate) + 'Cate_monthlyNumber'].fillna(0)
    return jdata_train_df, jdata_test_df

print(len(jdata_train_df))
print(len(jdata_train_df.columns.values))
print(len(jdata_test_df))
print(len(jdata_test_df.columns.values))
jdata_train_df, jdata_test_df = getUserCateMonthlyBuyNumber(jdata_train_df, jdata_test_df, jdata_user_order_data, 30)
jdata_train_df, jdata_test_df = getUserCateMonthlyBuyNumber(jdata_train_df, jdata_test_df, jdata_user_order_data, 101)
print(len(jdata_train_df))
print(len(jdata_train_df.columns.values))
print(len(jdata_test_df))
print(len(jdata_test_df.columns.values))


1088164
41
98924
39
1088164
43
98924
41


In [26]:
# 统计某用户月平均下单目标类目的次数
def getUserTargetCateMonthlyOrderNumber(jdata_train_df, jdata_test_df, jdata_user_order_data):
    jdata_user_order_data_1 = jdata_user_order_data[['user_id', 'o_sku_num']][(jdata_user_order_data.cate == 101) | (jdata_user_order_data.cate == 30)]
    jdata_user_order_data_pivot_table = pd.pivot_table(jdata_user_order_data_1, index=['user_id'], values=['o_sku_num'], aggfunc=len)
    jdata_user_order_data_pivot_table.rename(columns={'o_sku_num':'user_targetCate_monthlyOrderNumber'}, inplace=True)
    jdata_user_order_data_pivot_table.reset_index(inplace=True)
    jdata_user_order_data_pivot_table['user_targetCate_monthlyOrderNumber'] = jdata_user_order_data_pivot_table['user_targetCate_monthlyOrderNumber'] / 12
    jdata_train_df = pd.merge(jdata_train_df, jdata_user_order_data_pivot_table[['user_id', 'user_targetCate_monthlyOrderNumber']], on=['user_id'], how='left')
    jdata_test_df = pd.merge(jdata_test_df, jdata_user_order_data_pivot_table[['user_id', 'user_targetCate_monthlyOrderNumber']], on=['user_id'], how='left')
    jdata_train_df['user_targetCate_monthlyOrderNumber'] = jdata_train_df['user_targetCate_monthlyOrderNumber'].fillna(0)
    jdata_test_df['user_targetCate_monthlyOrderNumber'] = jdata_test_df['user_targetCate_monthlyOrderNumber'].fillna(0)
    return jdata_train_df, jdata_test_df

print(len(jdata_train_df))
print(len(jdata_train_df.columns.values))
print(len(jdata_test_df))
print(len(jdata_test_df.columns.values))
jdata_train_df, jdata_test_df = getUserTargetCateMonthlyOrderNumber(jdata_train_df, jdata_test_df, jdata_user_order_data)
print(len(jdata_train_df))
print(len(jdata_train_df.columns.values))
print(len(jdata_test_df))
print(len(jdata_test_df.columns.values))


1088164
43
98924
41
1088164
44
98924
42


In [27]:
# 统计某用户月平均下单某个类目的次数
def getUserCateMonthlyOrderNumber(jdata_train_df, jdata_test_df, jdata_user_order_data, cate):
    jdata_user_order_data_1 = jdata_user_order_data[['user_id', 'o_sku_num']][(jdata_user_order_data.cate == cate)]
    jdata_user_order_data_pivot_table = pd.pivot_table(jdata_user_order_data_1, index=['user_id'], values=['o_sku_num'], aggfunc=len)
    jdata_user_order_data_pivot_table.rename(columns={'o_sku_num':'user_' + str(cate) + 'Cate_monthlyOrderNumber'}, inplace=True)
    jdata_user_order_data_pivot_table.reset_index(inplace=True)
    jdata_user_order_data_pivot_table['user_' + str(cate) + 'Cate_monthlyOrderNumber'] = jdata_user_order_data_pivot_table['user_' + str(cate) + 'Cate_monthlyOrderNumber'] / 12
    jdata_train_df = pd.merge(jdata_train_df, jdata_user_order_data_pivot_table[['user_id', 'user_' + str(cate) + 'Cate_monthlyOrderNumber']], on=['user_id'], how='left')
    jdata_test_df = pd.merge(jdata_test_df, jdata_user_order_data_pivot_table[['user_id', 'user_' + str(cate) + 'Cate_monthlyOrderNumber']], on=['user_id'], how='left')
    jdata_train_df['user_' + str(cate) + 'Cate_monthlyOrderNumber'] = jdata_train_df['user_' + str(cate) + 'Cate_monthlyOrderNumber'].fillna(0)
    jdata_test_df['user_' + str(cate) + 'Cate_monthlyOrderNumber'] = jdata_test_df['user_' + str(cate) + 'Cate_monthlyOrderNumber'].fillna(0)
    return jdata_train_df, jdata_test_df

print(len(jdata_train_df))
print(len(jdata_train_df.columns.values))
print(len(jdata_test_df))
print(len(jdata_test_df.columns.values))
jdata_train_df, jdata_test_df = getUserCateMonthlyOrderNumber(jdata_train_df, jdata_test_df, jdata_user_order_data, 30)
jdata_train_df, jdata_test_df = getUserCateMonthlyOrderNumber(jdata_train_df, jdata_test_df, jdata_user_order_data, 101)
print(len(jdata_train_df))
print(len(jdata_train_df.columns.values))
print(len(jdata_test_df))
print(len(jdata_test_df.columns.values))


1088164
44
98924
42
1088164
46
98924
44


In [28]:
# 统计某用户月平均点击目标类目商品的个数
def getUserTargetCateMonthlyClickNumber(jdata_train_df, jdata_test_df, jdata_user_action_data):
    jdata_user_action_data_1 = jdata_user_action_data[['user_id', 'a_num']][((jdata_user_action_data.cate == 101) | (jdata_user_action_data.cate == 30)) & (jdata_user_action_data.a_type == 1)]
    jdata_user_action_data_pivot_table = pd.pivot_table(jdata_user_action_data_1, index=['user_id'], values=['a_num'], aggfunc=sum)
    jdata_user_action_data_pivot_table.rename(columns={'a_num':'user_targetCate_monthlyClickNumber'}, inplace=True)
    jdata_user_action_data_pivot_table.reset_index(inplace=True)
    jdata_user_action_data_pivot_table['user_targetCate_monthlyClickNumber'] = jdata_user_action_data_pivot_table['user_targetCate_monthlyClickNumber'] / 12
    jdata_train_df = pd.merge(jdata_train_df, jdata_user_action_data_pivot_table[['user_id', 'user_targetCate_monthlyClickNumber']], on=['user_id'], how='left')
    jdata_test_df = pd.merge(jdata_test_df, jdata_user_action_data_pivot_table[['user_id', 'user_targetCate_monthlyClickNumber']], on=['user_id'], how='left')
    jdata_train_df['user_targetCate_monthlyClickNumber'] = jdata_train_df['user_targetCate_monthlyClickNumber'].fillna(0)
    jdata_test_df['user_targetCate_monthlyClickNumber'] = jdata_test_df['user_targetCate_monthlyClickNumber'].fillna(0)
    return jdata_train_df, jdata_test_df

print(len(jdata_train_df))
print(len(jdata_train_df.columns.values))
print(len(jdata_test_df))
print(len(jdata_test_df.columns.values))
jdata_train_df, jdata_test_df = getUserTargetCateMonthlyClickNumber(jdata_train_df, jdata_test_df, jdata_user_action_data)
print(len(jdata_train_df))
print(len(jdata_train_df.columns.values))
print(len(jdata_test_df))
print(len(jdata_test_df.columns.values))


1088164
46
98924
44
1088164
47
98924
45


In [29]:
# 统计某用户月平均点击某个类目商品的个数
def getUserCateMonthlyClickNumber(jdata_train_df, jdata_test_df, jdata_user_action_data, cate):
    jdata_user_action_data_1 = jdata_user_action_data[['user_id', 'a_num']][((jdata_user_action_data.cate == cate)) & (jdata_user_action_data.a_type == 1)]
    jdata_user_action_data_pivot_table = pd.pivot_table(jdata_user_action_data_1, index=['user_id'], values=['a_num'], aggfunc=sum)
    jdata_user_action_data_pivot_table.rename(columns={'a_num':'user_' + str(cate) + 'Cate_monthlyClickNumber'}, inplace=True)
    jdata_user_action_data_pivot_table.reset_index(inplace=True)
    jdata_user_action_data_pivot_table['user_' + str(cate) + 'Cate_monthlyClickNumber'] = jdata_user_action_data_pivot_table['user_' + str(cate) + 'Cate_monthlyClickNumber'] / 12
    jdata_train_df = pd.merge(jdata_train_df, jdata_user_action_data_pivot_table[['user_id', 'user_' + str(cate) + 'Cate_monthlyClickNumber']], on=['user_id'], how='left')
    jdata_test_df = pd.merge(jdata_test_df, jdata_user_action_data_pivot_table[['user_id', 'user_' + str(cate) + 'Cate_monthlyClickNumber']], on=['user_id'], how='left')
    jdata_train_df['user_' + str(cate) + 'Cate_monthlyClickNumber'] = jdata_train_df['user_' + str(cate) + 'Cate_monthlyClickNumber'].fillna(0)
    jdata_test_df['user_' + str(cate) + 'Cate_monthlyClickNumber'] = jdata_test_df['user_' + str(cate) + 'Cate_monthlyClickNumber'].fillna(0)
    return jdata_train_df, jdata_test_df

print(len(jdata_train_df))
print(len(jdata_train_df.columns.values))
print(len(jdata_test_df))
print(len(jdata_test_df.columns.values))
jdata_train_df, jdata_test_df = getUserCateMonthlyClickNumber(jdata_train_df, jdata_test_df, jdata_user_action_data, 30)
jdata_train_df, jdata_test_df = getUserCateMonthlyClickNumber(jdata_train_df, jdata_test_df, jdata_user_action_data, 101)
print(len(jdata_train_df))
print(len(jdata_train_df.columns.values))
print(len(jdata_test_df))
print(len(jdata_test_df.columns.values))


1088164
47
98924
45
1088164
49
98924
47


In [30]:
# 统计某用户月平均下单相关类目商品的个数
def getUserRelatedCateMonthlyBuyNumber(jdata_train_df, jdata_test_df, jdata_user_order_data):
    jdata_user_order_data_1 = jdata_user_order_data[['user_id', 'o_sku_num']][(jdata_user_order_data.cate != 101) & (jdata_user_order_data.cate != 30)]
    jdata_user_order_data_pivot_table = pd.pivot_table(jdata_user_order_data_1, index=['user_id'], values=['o_sku_num'], aggfunc=sum)
    jdata_user_order_data_pivot_table.rename(columns={'o_sku_num':'user_relatedCate_monthlyNumber'}, inplace=True)
    jdata_user_order_data_pivot_table.reset_index(inplace=True)
    jdata_user_order_data_pivot_table['user_relatedCate_monthlyNumber'] = jdata_user_order_data_pivot_table['user_relatedCate_monthlyNumber'] / 12
    jdata_train_df = pd.merge(jdata_train_df, jdata_user_order_data_pivot_table[['user_id', 'user_relatedCate_monthlyNumber']], on=['user_id'], how='left')
    jdata_test_df = pd.merge(jdata_test_df, jdata_user_order_data_pivot_table[['user_id', 'user_relatedCate_monthlyNumber']], on=['user_id'], how='left')
    jdata_train_df['user_relatedCate_monthlyNumber'] = jdata_train_df['user_relatedCate_monthlyNumber'].fillna(0)
    jdata_test_df['user_relatedCate_monthlyNumber'] = jdata_test_df['user_relatedCate_monthlyNumber'].fillna(0)
    return jdata_train_df, jdata_test_df
    
jdata_train_df, jdata_test_df = getUserRelatedCateMonthlyBuyNumber(jdata_train_df, jdata_test_df, jdata_user_order_data)


In [31]:
# 统计某用户月平均下单相关类目的次数
def getUserRelatedCateMonthlyOrderNumber(jdata_train_df, jdata_test_df, jdata_user_order_data):
    jdata_user_order_data_1 = jdata_user_order_data[['user_id', 'o_sku_num']][(jdata_user_order_data.cate != 101) & (jdata_user_order_data.cate != 30)]
    jdata_user_order_data_pivot_table = pd.pivot_table(jdata_user_order_data_1, index=['user_id'], values=['o_sku_num'], aggfunc=len)
    jdata_user_order_data_pivot_table.rename(columns={'o_sku_num':'user_relatedCate_monthlyOrderNumber'}, inplace=True)
    jdata_user_order_data_pivot_table.reset_index(inplace=True)
    jdata_user_order_data_pivot_table['user_relatedCate_monthlyOrderNumber'] = jdata_user_order_data_pivot_table['user_relatedCate_monthlyOrderNumber'] / 12
    jdata_train_df = pd.merge(jdata_train_df, jdata_user_order_data_pivot_table[['user_id', 'user_relatedCate_monthlyOrderNumber']], on=['user_id'], how='left')
    jdata_test_df = pd.merge(jdata_test_df, jdata_user_order_data_pivot_table[['user_id', 'user_relatedCate_monthlyOrderNumber']], on=['user_id'], how='left')
    jdata_train_df['user_relatedCate_monthlyOrderNumber'] = jdata_train_df['user_relatedCate_monthlyOrderNumber'].fillna(0)
    jdata_test_df['user_relatedCate_monthlyOrderNumber'] = jdata_test_df['user_relatedCate_monthlyOrderNumber'].fillna(0)
    return jdata_train_df, jdata_test_df

print(len(jdata_train_df))
print(len(jdata_train_df.columns.values))
print(len(jdata_test_df))
print(len(jdata_test_df.columns.values))
jdata_train_df, jdata_test_df = getUserRelatedCateMonthlyOrderNumber(jdata_train_df, jdata_test_df, jdata_user_order_data)
print(len(jdata_train_df))
print(len(jdata_train_df.columns.values))
print(len(jdata_test_df))
print(len(jdata_test_df.columns.values))


1088164
50
98924
48
1088164
51
98924
49


In [32]:
# 统计某用户月平均点击相关类目商品的个数
def getUserRelatedCateMonthlyClickNumber(jdata_train_df, jdata_test_df, jdata_user_action_data):
    jdata_user_action_data_1 = jdata_user_action_data[['user_id', 'a_num']][((jdata_user_action_data.cate != 101) & (jdata_user_action_data.cate != 30)) & (jdata_user_action_data.a_type == 1)]
    jdata_user_action_data_pivot_table = pd.pivot_table(jdata_user_action_data_1, index=['user_id'], values=['a_num'], aggfunc=sum)
    jdata_user_action_data_pivot_table.rename(columns={'a_num':'user_relatedCate_monthlyClickNumber'}, inplace=True)
    jdata_user_action_data_pivot_table.reset_index(inplace=True)
    jdata_user_action_data_pivot_table['user_relatedCate_monthlyClickNumber'] = jdata_user_action_data_pivot_table['user_relatedCate_monthlyClickNumber'] / 12
    jdata_train_df = pd.merge(jdata_train_df, jdata_user_action_data_pivot_table[['user_id', 'user_relatedCate_monthlyClickNumber']], on=['user_id'], how='left')
    jdata_test_df = pd.merge(jdata_test_df, jdata_user_action_data_pivot_table[['user_id', 'user_relatedCate_monthlyClickNumber']], on=['user_id'], how='left')
    jdata_train_df['user_relatedCate_monthlyClickNumber'] = jdata_train_df['user_relatedCate_monthlyClickNumber'].fillna(0)
    jdata_test_df['user_relatedCate_monthlyClickNumber'] = jdata_test_df['user_relatedCate_monthlyClickNumber'].fillna(0)
    return jdata_train_df, jdata_test_df

print(len(jdata_train_df))
print(len(jdata_train_df.columns.values))
print(len(jdata_test_df))
print(len(jdata_test_df.columns.values))
jdata_train_df, jdata_test_df = getUserRelatedCateMonthlyClickNumber(jdata_train_df, jdata_test_df, jdata_user_action_data)
print(len(jdata_train_df))
print(len(jdata_train_df.columns.values))
print(len(jdata_test_df))
print(len(jdata_test_df.columns.values))


1088164
51
98924
49
1088164
52
98924
50


In [33]:
# 定义获取用户平均每个月购买/点击目标类目商品个数占所有类目的比例特征
def getTargetCateRate(jdata_train_df, jdata_test_df):
    jdata_train_df['user_targetCate_monthlyBuyRate'] = jdata_train_df['user_targetCate_monthlyNumber'] / (jdata_train_df['user_targetCate_monthlyNumber'] + jdata_train_df['user_relatedCate_monthlyNumber'])
    jdata_test_df['user_targetCate_monthlyBuyRate'] = jdata_test_df['user_targetCate_monthlyNumber'] / (jdata_test_df['user_targetCate_monthlyNumber'] + jdata_test_df['user_relatedCate_monthlyNumber'])
    jdata_train_df['user_targetCate_monthlyClickRate'] = jdata_train_df['user_targetCate_monthlyClickNumber'] / (jdata_train_df['user_targetCate_monthlyClickNumber'] + jdata_train_df['user_relatedCate_monthlyClickNumber'])
    jdata_test_df['user_targetCate_monthlyClickRate'] = jdata_test_df['user_targetCate_monthlyClickNumber'] / (jdata_test_df['user_targetCate_monthlyClickNumber'] + jdata_test_df['user_relatedCate_monthlyClickNumber'])
    return jdata_train_df, jdata_test_df

print(len(jdata_train_df))
print(len(jdata_train_df.columns.values))
print(len(jdata_test_df))
print(len(jdata_test_df.columns.values))
jdata_train_df, jdata_test_df = getTargetCateRate(jdata_train_df, jdata_test_df)
print(len(jdata_train_df))
print(len(jdata_train_df.columns.values))
print(len(jdata_test_df))
print(len(jdata_test_df.columns.values))
    

1088164
52
98924
50
1088164
54
98924
52


In [34]:
# 定义获取用户历史好评率和差评率
def getUserCommentFuture(jdata_train_df, jdata_test_df, jdata_user_comment_score_data):
    jdata_user_comment_score_data_sum_pivot_table = pd.pivot_table(jdata_user_comment_score_data, index=['user_id'], values=['score_level'], aggfunc=len)
    jdata_user_comment_score_data_sum_pivot_table.rename(columns={'score_level':'user_comment_count'}, inplace=True)
    jdata_user_comment_score_data_sum_pivot_table.reset_index(inplace=True)
    jdata_train_df = pd.merge(jdata_train_df, jdata_user_comment_score_data_sum_pivot_table, on=['user_id'], how='left')
    jdata_test_df = pd.merge(jdata_test_df, jdata_user_comment_score_data_sum_pivot_table, on=['user_id'], how='left')

    jdata_user_comment_score_data_praise_pivot_table = pd.pivot_table(jdata_user_comment_score_data[jdata_user_comment_score_data.score_level == 1], index=['user_id'], values=['score_level'], aggfunc=len)
    jdata_user_comment_score_data_praise_pivot_table.rename(columns={'score_level':'user_praiseComment_count'}, inplace=True)
    jdata_user_comment_score_data_praise_pivot_table.reset_index(inplace=True)
    jdata_train_df = pd.merge(jdata_train_df, jdata_user_comment_score_data_praise_pivot_table, on=['user_id'], how='left')
    jdata_test_df = pd.merge(jdata_test_df, jdata_user_comment_score_data_praise_pivot_table, on=['user_id'], how='left')

    jdata_user_comment_score_data_bad_pivot_table = pd.pivot_table(jdata_user_comment_score_data[jdata_user_comment_score_data.score_level == 3], index=['user_id'], values=['score_level'], aggfunc=len)
    jdata_user_comment_score_data_bad_pivot_table.rename(columns={'score_level':'user_badComment_count'}, inplace=True)
    jdata_user_comment_score_data_bad_pivot_table.reset_index(inplace=True)
    jdata_train_df = pd.merge(jdata_train_df, jdata_user_comment_score_data_bad_pivot_table, on=['user_id'], how='left')
    jdata_test_df = pd.merge(jdata_test_df, jdata_user_comment_score_data_bad_pivot_table, on=['user_id'], how='left')

    jdata_train_df['user_praiseComment_rate'] = jdata_train_df['user_praiseComment_count'] / jdata_train_df['user_comment_count']
    jdata_train_df['user_badComment_rate'] = jdata_train_df['user_badComment_count'] / jdata_train_df['user_comment_count']
    jdata_train_df = jdata_train_df.fillna(0)
    jdata_test_df['user_praiseComment_rate'] = jdata_test_df['user_praiseComment_count'] / jdata_test_df['user_comment_count']
    jdata_test_df['user_badComment_rate'] = jdata_test_df['user_badComment_count'] / jdata_test_df['user_comment_count']
    jdata_test_df = jdata_test_df.fillna(0)
    return jdata_train_df, jdata_test_df

print(len(jdata_train_df))
print(len(jdata_train_df.columns.values))
print(len(jdata_test_df))
print(len(jdata_test_df.columns.values))
jdata_train_df, jdata_test_df = getUserCommentFuture(jdata_train_df, jdata_test_df, jdata_user_comment_score_data)
print(len(jdata_train_df))
print(len(jdata_train_df.columns.values))
print(len(jdata_test_df))
print(len(jdata_test_df.columns.values))



1088164
54
98924
52
1088164
59
98924
57


In [35]:
# 统计用户对某个类目月下单的统计特征
def getUserCateOrderDayFuture(jdata_train_df, jdata_test_df, jdata_user_order_data, cate):
    jdata_user_order_data_copy = jdata_user_order_data[jdata_user_order_data.cate == cate]
    jdata_user_order_data_copy = jdata_user_order_data_copy.drop_duplicates(['user_id', 'cate', 'o_date'])
    jdata_user_order_data_copy_pivot_table = pd.pivot_table(jdata_user_order_data_copy, index=['user_id'], values=['day'], aggfunc={min, max, mean})
    jdata_user_order_data_copy_pivot_table.reset_index(inplace=True)
    jdata_user_order_data_copy_pivot_table.columns = ['user_id', 'user_cate' + str(cate) + '_orderDayMax', 'user_cate' + str(cate) + '_orderDayMean', 'user_cate' + str(cate) + '_orderDayMin']
    jdata_train_df = pd.merge(jdata_train_df, jdata_user_order_data_copy_pivot_table, on=['user_id'], how='left')
    jdata_test_df = pd.merge(jdata_test_df, jdata_user_order_data_copy_pivot_table, on=['user_id'], how='left')
    return jdata_train_df, jdata_test_df

print(len(jdata_train_df))
print(len(jdata_train_df.columns.values))
print(len(jdata_test_df))
print(len(jdata_test_df.columns.values))
jdata_train_df, jdata_test_df = getUserCateOrderDayFuture(jdata_train_df, jdata_test_df, jdata_user_order_data, 101)
jdata_train_df, jdata_test_df = getUserCateOrderDayFuture(jdata_train_df, jdata_test_df, jdata_user_order_data, 30)
print(len(jdata_train_df))
print(len(jdata_train_df.columns.values))
print(len(jdata_test_df))
print(len(jdata_test_df.columns.values))


1088164
59
98924
57
1088164
65
98924
63


In [36]:
# 统计用户对目标类目月下单的统计特征
def getUserTargetCateOrderDayFuture(jdata_train_df, jdata_test_df, jdata_user_order_data):
    jdata_user_order_data_copy = jdata_user_order_data[(jdata_user_order_data.cate == 101) | (jdata_user_order_data.cate == 30)]
    jdata_user_order_data_copy = jdata_user_order_data_copy.drop_duplicates(['user_id', 'cate', 'o_date'])
    jdata_user_order_data_copy_pivot_table = pd.pivot_table(jdata_user_order_data_copy, index=['user_id'], values=['day'], aggfunc={min, max, mean})
    jdata_user_order_data_copy_pivot_table.reset_index(inplace=True)
    jdata_user_order_data_copy_pivot_table.columns = ['user_id', 'user_targetCate_orderDayMax', 'user_targetCate_orderDayMean', 'user_targetCate_orderDayMin']
    jdata_train_df = pd.merge(jdata_train_df, jdata_user_order_data_copy_pivot_table, on=['user_id'], how='left')
    jdata_test_df = pd.merge(jdata_test_df, jdata_user_order_data_copy_pivot_table, on=['user_id'], how='left')
    return jdata_train_df, jdata_test_df

print(len(jdata_train_df))
print(len(jdata_train_df.columns.values))
print(len(jdata_test_df))
print(len(jdata_test_df.columns.values))
jdata_train_df, jdata_test_df = getUserTargetCateOrderDayFuture(jdata_train_df, jdata_test_df, jdata_user_order_data)
print(len(jdata_train_df))
print(len(jdata_train_df.columns.values))
print(len(jdata_test_df))
print(len(jdata_test_df.columns.values))


1088164
65
98924
63
1088164
68
98924
66


In [37]:
# 统计用户对相关类目月下单的统计特征
def getUserRelatedCateOrderDayFuture(jdata_train_df, jdata_test_df, jdata_user_order_data):
    jdata_user_order_data_copy = jdata_user_order_data[(jdata_user_order_data.cate != 101) & (jdata_user_order_data.cate != 30)]
    jdata_user_order_data_copy = jdata_user_order_data_copy.drop_duplicates(['user_id', 'cate', 'o_date'])
    jdata_user_order_data_copy_pivot_table = pd.pivot_table(jdata_user_order_data_copy, index=['user_id'], values=['day'], aggfunc={min, max, mean})
    jdata_user_order_data_copy_pivot_table.reset_index(inplace=True)
    jdata_user_order_data_copy_pivot_table.columns = ['user_id', 'user_relatedCate_orderDayMax', 'user_relatedCate_orderDayMean', 'user_relatedCate_orderDayMin']
    jdata_train_df = pd.merge(jdata_train_df, jdata_user_order_data_copy_pivot_table, on=['user_id'], how='left')
    jdata_test_df = pd.merge(jdata_test_df, jdata_user_order_data_copy_pivot_table, on=['user_id'], how='left')
    return jdata_train_df, jdata_test_df

print(len(jdata_train_df))
print(len(jdata_train_df.columns.values))
print(len(jdata_test_df))
print(len(jdata_test_df.columns.values))
jdata_train_df, jdata_test_df = getUserRelatedCateOrderDayFuture(jdata_train_df, jdata_test_df, jdata_user_order_data)
print(len(jdata_train_df))
print(len(jdata_train_df.columns.values))
print(len(jdata_test_df))
print(len(jdata_test_df.columns.values))


1088164
68
98924
66
1088164
71
98924
69


In [38]:
# 统计某用户上个月关注某个类目商品的次数，逐个月进行处理
def getUserCateLastMonthFocusNumberMonthly(jdata_df, jdata_user_action_data, year, month, cate):
    jdata_train_df_dealMonth = jdata_df
    jdata_user_action_data_focus_future = jdata_user_action_data[(jdata_user_action_data.month == month) & (jdata_user_action_data.year == year) & (jdata_user_action_data.a_type == 2) & (jdata_user_action_data.cate == cate)]
    jdata_user_action_data_focus_future_pivot_table = pd.pivot_table(jdata_user_action_data_focus_future, index=['user_id'], values=['a_num'], aggfunc=sum)
    jdata_user_action_data_focus_future_pivot_table.reset_index(inplace=True)
    jdata_user_action_data_focus_future_pivot_table.rename(columns={'a_num':'user_cate'+ str(cate) + '_lastMonth_focusNumber'}, inplace=True)
    jdata_train_df_dealMonth = pd.merge(jdata_train_df_dealMonth, jdata_user_action_data_focus_future_pivot_table, on=['user_id'], how='left')
    jdata_train_df_dealMonth['user_cate'+ str(cate) + '_lastMonth_focusNumber'] = jdata_train_df_dealMonth['user_cate'+ str(cate) + '_lastMonth_focusNumber'].fillna(0)
    return jdata_train_df_dealMonth


def getUserCateLastMonthFocusNumber(jdata_train_df, jdata_test_df, jdata_user_action_data, cate):
    jdata_train_df_temp = pd.DataFrame()
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLastMonthFocusNumberMonthly(jdata_train_df[(jdata_train_df.month == 4) & (jdata_train_df.year == 2017)], jdata_user_action_data, 2017, 3, cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLastMonthFocusNumberMonthly(jdata_train_df[(jdata_train_df.month == 3) & (jdata_train_df.year == 2017)], jdata_user_action_data, 2017, 2, cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLastMonthFocusNumberMonthly(jdata_train_df[(jdata_train_df.month == 2) & (jdata_train_df.year == 2017)], jdata_user_action_data, 2017, 1, cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLastMonthFocusNumberMonthly(jdata_train_df[(jdata_train_df.month == 1) & (jdata_train_df.year == 2017)], jdata_user_action_data, 2016, 12, cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLastMonthFocusNumberMonthly(jdata_train_df[(jdata_train_df.month == 12) & (jdata_train_df.year == 2016)], jdata_user_action_data, 2016, 11, cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLastMonthFocusNumberMonthly(jdata_train_df[(jdata_train_df.month == 11) & (jdata_train_df.year == 2016)], jdata_user_action_data, 2016, 10, cate)])
    jdata_test_df = getUserCateLastMonthFocusNumberMonthly(jdata_test_df, jdata_user_action_data, 2017, 4, cate)
    return jdata_train_df_temp, jdata_test_df

print(len(jdata_train_df))
print(len(jdata_train_df.columns.values))
print(len(jdata_test_df))
print(len(jdata_test_df.columns.values))
jdata_train_df_huachuang, jdata_test_df = getUserCateLastMonthFocusNumber(jdata_train_df, jdata_test_df, jdata_user_action_data, 101)
jdata_train_df_huachuang, jdata_test_df = getUserCateLastMonthFocusNumber(jdata_train_df_huachuang, jdata_test_df, jdata_user_action_data, 30)
jdata_train_df_huachuang, jdata_test_df = getUserCateLastMonthFocusNumber(jdata_train_df_huachuang, jdata_test_df, jdata_user_action_data, 1)
jdata_train_df_huachuang, jdata_test_df = getUserCateLastMonthFocusNumber(jdata_train_df_huachuang, jdata_test_df, jdata_user_action_data, 71)
jdata_train_df_huachuang, jdata_test_df = getUserCateLastMonthFocusNumber(jdata_train_df_huachuang, jdata_test_df, jdata_user_action_data, 46)
jdata_train_df_huachuang, jdata_test_df = getUserCateLastMonthFocusNumber(jdata_train_df_huachuang, jdata_test_df, jdata_user_action_data, 83)
print(len(jdata_train_df_huachuang))
print(len(jdata_train_df_huachuang.columns.values))
print(len(jdata_test_df))
print(len(jdata_test_df.columns.values))


1088164
71
98924
69
593544
77
98924
75


In [39]:
# 统计某用户3个月关注某个类目商品的次数，逐个月进行处理
def getUserCateLast3MonthFocusNumberMonthly(jdata_df, jdata_user_action_data, start_time, end_time, cate):
    jdata_train_df_dealMonth = jdata_df
    jdata_user_action_data_focus_future = jdata_user_action_data[(jdata_user_action_data.a_date >= start_time) & (jdata_user_action_data.a_date < end_time) & (jdata_user_action_data.a_type == 2) & (jdata_user_action_data.cate == cate)]
    jdata_user_action_data_focus_future_pivot_table = pd.pivot_table(jdata_user_action_data_focus_future, index=['user_id'], values=['a_num'], aggfunc=sum)
    jdata_user_action_data_focus_future_pivot_table.reset_index(inplace=True)
    jdata_user_action_data_focus_future_pivot_table.rename(columns={'a_num':'user_cate'+ str(cate) + '_last3Month_focusNumber'}, inplace=True)
    jdata_train_df_dealMonth = pd.merge(jdata_train_df_dealMonth, jdata_user_action_data_focus_future_pivot_table, on=['user_id'], how='left')
    jdata_train_df_dealMonth['user_cate'+ str(cate) + '_last3Month_focusNumber'] = jdata_train_df_dealMonth['user_cate'+ str(cate) + '_last3Month_focusNumber'].fillna(0)
    return jdata_train_df_dealMonth

def getUserCateLast3MonthFocusNumber(jdata_train_df, jdata_test_df, jdata_user_action_data, cate):
    jdata_train_df_temp = pd.DataFrame()
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLast3MonthFocusNumberMonthly(jdata_train_df[(jdata_train_df.month == 4) & (jdata_train_df.year == 2017)], jdata_user_action_data, datetime.datetime(2017, 1, 1), datetime.datetime(2017, 4, 1), cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLast3MonthFocusNumberMonthly(jdata_train_df[(jdata_train_df.month == 3) & (jdata_train_df.year == 2017)], jdata_user_action_data, datetime.datetime(2016, 12, 1), datetime.datetime(2017, 3, 1), cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLast3MonthFocusNumberMonthly(jdata_train_df[(jdata_train_df.month == 2) & (jdata_train_df.year == 2017)], jdata_user_action_data, datetime.datetime(2016, 11, 1), datetime.datetime(2017, 2, 1), cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLast3MonthFocusNumberMonthly(jdata_train_df[(jdata_train_df.month == 1) & (jdata_train_df.year == 2017)], jdata_user_action_data, datetime.datetime(2016, 10, 1), datetime.datetime(2017, 1, 1), cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLast3MonthFocusNumberMonthly(jdata_train_df[(jdata_train_df.month == 12) & (jdata_train_df.year == 2016)], jdata_user_action_data, datetime.datetime(2016, 9, 1), datetime.datetime(2016, 12, 1), cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLast3MonthFocusNumberMonthly(jdata_train_df[(jdata_train_df.month == 11) & (jdata_train_df.year == 2016)], jdata_user_action_data, datetime.datetime(2016, 8, 1), datetime.datetime(2016, 11, 1), cate)])
    jdata_test_df = getUserCateLast3MonthFocusNumberMonthly(jdata_test_df, jdata_user_action_data, datetime.datetime(2017, 2, 1), datetime.datetime(2017, 5, 1), cate)
    return jdata_train_df_temp, jdata_test_df

print(len(jdata_train_df_huachuang))
print(len(jdata_train_df_huachuang.columns.values))
print(len(jdata_test_df))
print(len(jdata_test_df.columns.values))
jdata_train_df_huachuang, jdata_test_df = getUserCateLast3MonthFocusNumber(jdata_train_df_huachuang, jdata_test_df, jdata_user_action_data, 101)
jdata_train_df_huachuang, jdata_test_df = getUserCateLast3MonthFocusNumber(jdata_train_df_huachuang, jdata_test_df, jdata_user_action_data, 30)
jdata_train_df_huachuang, jdata_test_df = getUserCateLast3MonthFocusNumber(jdata_train_df_huachuang, jdata_test_df, jdata_user_action_data, 1)
jdata_train_df_huachuang, jdata_test_df = getUserCateLast3MonthFocusNumber(jdata_train_df_huachuang, jdata_test_df, jdata_user_action_data, 71)
jdata_train_df_huachuang, jdata_test_df = getUserCateLast3MonthFocusNumber(jdata_train_df_huachuang, jdata_test_df, jdata_user_action_data, 46)
jdata_train_df_huachuang, jdata_test_df = getUserCateLast3MonthFocusNumber(jdata_train_df_huachuang, jdata_test_df, jdata_user_action_data, 83)
print(len(jdata_train_df_huachuang))
print(len(jdata_train_df_huachuang.columns.values))
print(len(jdata_test_df))
print(len(jdata_test_df.columns.values))


593544
77
98924
75
593544
83
98924
81


In [40]:
# 统计某用户上个月浏览某个类目商品的次数，逐个月进行处理
def getUserCateLastMonthClickNumberMonthly(jdata_df, jdata_user_action_data, year, month, cate):
    jdata_train_df_dealMonth = jdata_df
    jdata_user_action_data_click_future = jdata_user_action_data[(jdata_user_action_data.month == month) & (jdata_user_action_data.year == year) & (jdata_user_action_data.a_type == 1) & (jdata_user_action_data.cate == cate)]
    jdata_user_action_data_click_future_pivot_table = pd.pivot_table(jdata_user_action_data_click_future, index=['user_id'], values=['a_num'], aggfunc=sum)
    jdata_user_action_data_click_future_pivot_table.reset_index(inplace=True)
    jdata_user_action_data_click_future_pivot_table.rename(columns={'a_num':'user_cate'+ str(cate) + '_lastMonth_clickNumber'}, inplace=True)
    jdata_train_df_dealMonth = pd.merge(jdata_train_df_dealMonth, jdata_user_action_data_click_future_pivot_table, on=['user_id'], how='left')
    jdata_train_df_dealMonth['user_cate'+ str(cate) + '_lastMonth_clickNumber'] = jdata_train_df_dealMonth['user_cate'+ str(cate) + '_lastMonth_clickNumber'].fillna(0)
    return jdata_train_df_dealMonth


def getUserCateLastMonthClickNumber(jdata_train_df, jdata_test_df, jdata_user_action_data, cate):
    jdata_train_df_temp = pd.DataFrame()
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLastMonthClickNumberMonthly(jdata_train_df[(jdata_train_df.month == 4) & (jdata_train_df.year == 2017)], jdata_user_action_data, 2017, 3, cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLastMonthClickNumberMonthly(jdata_train_df[(jdata_train_df.month == 3) & (jdata_train_df.year == 2017)], jdata_user_action_data, 2017, 2, cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLastMonthClickNumberMonthly(jdata_train_df[(jdata_train_df.month == 2) & (jdata_train_df.year == 2017)], jdata_user_action_data, 2017, 1, cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLastMonthClickNumberMonthly(jdata_train_df[(jdata_train_df.month == 1) & (jdata_train_df.year == 2017)], jdata_user_action_data, 2016, 12, cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLastMonthClickNumberMonthly(jdata_train_df[(jdata_train_df.month == 12) & (jdata_train_df.year == 2016)], jdata_user_action_data, 2016, 11, cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLastMonthClickNumberMonthly(jdata_train_df[(jdata_train_df.month == 11) & (jdata_train_df.year == 2016)], jdata_user_action_data, 2016, 10, cate)])
    jdata_test_df = getUserCateLastMonthClickNumberMonthly(jdata_test_df, jdata_user_action_data, 2017, 4, cate)
    return jdata_train_df_temp, jdata_test_df

print(len(jdata_train_df_huachuang))
print(len(jdata_train_df_huachuang.columns.values))
print(len(jdata_test_df))
print(len(jdata_test_df.columns.values))
jdata_train_df_huachuang, jdata_test_df = getUserCateLastMonthClickNumber(jdata_train_df_huachuang, jdata_test_df, jdata_user_action_data, 101)
jdata_train_df_huachuang, jdata_test_df = getUserCateLastMonthClickNumber(jdata_train_df_huachuang, jdata_test_df, jdata_user_action_data, 30)
jdata_train_df_huachuang, jdata_test_df = getUserCateLastMonthClickNumber(jdata_train_df_huachuang, jdata_test_df, jdata_user_action_data, 1)
jdata_train_df_huachuang, jdata_test_df = getUserCateLastMonthClickNumber(jdata_train_df_huachuang, jdata_test_df, jdata_user_action_data, 71)
jdata_train_df_huachuang, jdata_test_df = getUserCateLastMonthClickNumber(jdata_train_df_huachuang, jdata_test_df, jdata_user_action_data, 46)
jdata_train_df_huachuang, jdata_test_df = getUserCateLastMonthClickNumber(jdata_train_df_huachuang, jdata_test_df, jdata_user_action_data, 83)
print(len(jdata_train_df_huachuang))
print(len(jdata_train_df_huachuang.columns.values))
print(len(jdata_test_df))
print(len(jdata_test_df.columns.values))


593544
83
98924
81
593544
89
98924
87


In [41]:
# 统计某用户上3个月浏览某个类目商品的次数，逐个月进行处理
def getUserCateLast3MonthClickNumberMonthly(jdata_df, jdata_user_action_data, start_time, end_time, cate):
    jdata_train_df_dealMonth = jdata_df
    jdata_user_action_data_click_future = jdata_user_action_data[(jdata_user_action_data.a_date >= start_time) & (jdata_user_action_data.a_date < end_time) & (jdata_user_action_data.a_type == 1) & (jdata_user_action_data.cate == cate)]
    jdata_user_action_data_click_future_pivot_table = pd.pivot_table(jdata_user_action_data_click_future, index=['user_id'], values=['a_num'], aggfunc=sum)
    jdata_user_action_data_click_future_pivot_table.reset_index(inplace=True)
    jdata_user_action_data_click_future_pivot_table.rename(columns={'a_num':'user_cate'+ str(cate) + '_last3Month_clickNumber'}, inplace=True)
    jdata_train_df_dealMonth = pd.merge(jdata_train_df_dealMonth, jdata_user_action_data_click_future_pivot_table, on=['user_id'], how='left')
    jdata_train_df_dealMonth['user_cate'+ str(cate) + '_last3Month_clickNumber'] = jdata_train_df_dealMonth['user_cate'+ str(cate) + '_last3Month_clickNumber'].fillna(0)
    return jdata_train_df_dealMonth


def getUserCateLast3MonthClickNumber(jdata_train_df, jdata_test_df, jdata_user_action_data, cate):
    jdata_train_df_temp = pd.DataFrame()
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLast3MonthClickNumberMonthly(jdata_train_df[(jdata_train_df.month == 4) & (jdata_train_df.year == 2017)], jdata_user_action_data, datetime.datetime(2017, 1, 1), datetime.datetime(2017, 4, 1), cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLast3MonthClickNumberMonthly(jdata_train_df[(jdata_train_df.month == 3) & (jdata_train_df.year == 2017)], jdata_user_action_data, datetime.datetime(2016, 12, 1), datetime.datetime(2017, 3, 1), cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLast3MonthClickNumberMonthly(jdata_train_df[(jdata_train_df.month == 2) & (jdata_train_df.year == 2017)], jdata_user_action_data, datetime.datetime(2016, 11, 1), datetime.datetime(2017, 2, 1), cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLast3MonthClickNumberMonthly(jdata_train_df[(jdata_train_df.month == 1) & (jdata_train_df.year == 2017)], jdata_user_action_data, datetime.datetime(2016, 10, 1), datetime.datetime(2017, 1, 1), cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLast3MonthClickNumberMonthly(jdata_train_df[(jdata_train_df.month == 12) & (jdata_train_df.year == 2016)], jdata_user_action_data, datetime.datetime(2016, 9, 1), datetime.datetime(2016, 12, 1), cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLast3MonthClickNumberMonthly(jdata_train_df[(jdata_train_df.month == 11) & (jdata_train_df.year == 2016)], jdata_user_action_data, datetime.datetime(2016, 8, 1), datetime.datetime(2016, 11, 1), cate)])
    jdata_test_df = getUserCateLast3MonthClickNumberMonthly(jdata_test_df, jdata_user_action_data, datetime.datetime(2017, 2, 1), datetime.datetime(2017, 5, 1), cate)
    return jdata_train_df_temp, jdata_test_df

print(len(jdata_train_df_huachuang))
print(len(jdata_train_df_huachuang.columns.values))
print(len(jdata_test_df))
print(len(jdata_test_df.columns.values))
jdata_train_df_huachuang, jdata_test_df = getUserCateLast3MonthClickNumber(jdata_train_df_huachuang, jdata_test_df, jdata_user_action_data, 101)
jdata_train_df_huachuang, jdata_test_df = getUserCateLast3MonthClickNumber(jdata_train_df_huachuang, jdata_test_df, jdata_user_action_data, 30)
jdata_train_df_huachuang, jdata_test_df = getUserCateLast3MonthClickNumber(jdata_train_df_huachuang, jdata_test_df, jdata_user_action_data, 1)
jdata_train_df_huachuang, jdata_test_df = getUserCateLast3MonthClickNumber(jdata_train_df_huachuang, jdata_test_df, jdata_user_action_data, 71)
jdata_train_df_huachuang, jdata_test_df = getUserCateLast3MonthClickNumber(jdata_train_df_huachuang, jdata_test_df, jdata_user_action_data, 46)
jdata_train_df_huachuang, jdata_test_df = getUserCateLast3MonthClickNumber(jdata_train_df_huachuang, jdata_test_df, jdata_user_action_data, 83)
print(len(jdata_train_df_huachuang))
print(len(jdata_train_df_huachuang.columns.values))
print(len(jdata_test_df))
print(len(jdata_test_df.columns.values))


593544
89
98924
87
593544
95
98924
93


In [42]:
# 统计某用户上个月购买某个类目商品的个数，逐个月进行处理
def getUserCateLastMonthBuyNumberMonthly(jdata_df, jdata_user_order_data, year, month, cate):
    jdata_train_df_dealMonth = jdata_df
    jdata_user_order_data_buy_future = jdata_user_order_data[(jdata_user_order_data.month == month) & (jdata_user_order_data.year == year) & (jdata_user_order_data.cate == cate)]
    jdata_user_order_data_buy_future_pivot_table = pd.pivot_table(jdata_user_order_data_buy_future, index=['user_id'], values=['o_sku_num'], aggfunc=sum)
    jdata_user_order_data_buy_future_pivot_table.reset_index(inplace=True)
    jdata_user_order_data_buy_future_pivot_table.rename(columns={'o_sku_num':'user_cate'+ str(cate) + '_lastMonth_buyNumber'}, inplace=True)
    jdata_train_df_dealMonth = pd.merge(jdata_train_df_dealMonth, jdata_user_order_data_buy_future_pivot_table, on=['user_id'], how='left')
    jdata_train_df_dealMonth['user_cate'+ str(cate) + '_lastMonth_buyNumber'] = jdata_train_df_dealMonth['user_cate'+ str(cate) + '_lastMonth_buyNumber'].fillna(0)
    return jdata_train_df_dealMonth


def getUserCateLastMonthBuyNumber(jdata_train_df, jdata_test_df, jdata_user_order_data, cate):
    jdata_train_df_temp = pd.DataFrame()
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLastMonthBuyNumberMonthly(jdata_train_df[(jdata_train_df.month == 4) & (jdata_train_df.year == 2017)], jdata_user_order_data, 2017, 3, cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLastMonthBuyNumberMonthly(jdata_train_df[(jdata_train_df.month == 3) & (jdata_train_df.year == 2017)], jdata_user_order_data, 2017, 2, cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLastMonthBuyNumberMonthly(jdata_train_df[(jdata_train_df.month == 2) & (jdata_train_df.year == 2017)], jdata_user_order_data, 2017, 1, cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLastMonthBuyNumberMonthly(jdata_train_df[(jdata_train_df.month == 1) & (jdata_train_df.year == 2017)], jdata_user_order_data, 2016, 12, cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLastMonthBuyNumberMonthly(jdata_train_df[(jdata_train_df.month == 12) & (jdata_train_df.year == 2016)], jdata_user_order_data, 2016, 11, cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLastMonthBuyNumberMonthly(jdata_train_df[(jdata_train_df.month == 11) & (jdata_train_df.year == 2016)], jdata_user_order_data, 2016, 10, cate)])
    jdata_test_df = getUserCateLastMonthBuyNumberMonthly(jdata_test_df, jdata_user_order_data, 2017, 4, cate)
    return jdata_train_df_temp, jdata_test_df

print(len(jdata_train_df_huachuang))
print(len(jdata_train_df_huachuang.columns.values))
print(len(jdata_test_df))
print(len(jdata_test_df.columns.values))
jdata_train_df_huachuang, jdata_test_df = getUserCateLastMonthBuyNumber(jdata_train_df_huachuang, jdata_test_df, jdata_user_order_data, 101)
jdata_train_df_huachuang, jdata_test_df = getUserCateLastMonthBuyNumber(jdata_train_df_huachuang, jdata_test_df, jdata_user_order_data, 30)
jdata_train_df_huachuang, jdata_test_df = getUserCateLastMonthBuyNumber(jdata_train_df_huachuang, jdata_test_df, jdata_user_order_data, 1)
jdata_train_df_huachuang, jdata_test_df = getUserCateLastMonthBuyNumber(jdata_train_df_huachuang, jdata_test_df, jdata_user_order_data, 71)
jdata_train_df_huachuang, jdata_test_df = getUserCateLastMonthBuyNumber(jdata_train_df_huachuang, jdata_test_df, jdata_user_order_data, 46)
jdata_train_df_huachuang, jdata_test_df = getUserCateLastMonthBuyNumber(jdata_train_df_huachuang, jdata_test_df, jdata_user_order_data, 83)
print(len(jdata_train_df_huachuang))
print(len(jdata_train_df_huachuang.columns.values))
print(len(jdata_test_df))
print(len(jdata_test_df.columns.values))


593544
95
98924
93
593544
101
98924
99


In [43]:
# 统计某用户上3个月购买某个类目商品的个数，逐个月进行处理
def getUserCateLast3MonthBuyNumberMonthly(jdata_df, jdata_user_order_data, start_time, end_time, cate):
    jdata_train_df_dealMonth = jdata_df
    jdata_user_order_data_buy_future = jdata_user_order_data[(jdata_user_order_data.o_date >= start_time) & (jdata_user_order_data.o_date < end_time) & (jdata_user_order_data.cate == cate)]
    jdata_user_order_data_buy_future_pivot_table = pd.pivot_table(jdata_user_order_data_buy_future, index=['user_id'], values=['o_sku_num'], aggfunc=sum)
    jdata_user_order_data_buy_future_pivot_table.reset_index(inplace=True)
    jdata_user_order_data_buy_future_pivot_table.rename(columns={'o_sku_num':'user_cate'+ str(cate) + '_last3Month_buyNumber'}, inplace=True)
    jdata_train_df_dealMonth = pd.merge(jdata_train_df_dealMonth, jdata_user_order_data_buy_future_pivot_table, on=['user_id'], how='left')
    jdata_train_df_dealMonth['user_cate'+ str(cate) + '_last3Month_buyNumber'] = jdata_train_df_dealMonth['user_cate'+ str(cate) + '_last3Month_buyNumber'].fillna(0)
    return jdata_train_df_dealMonth


def getUserCateLast3MonthBuyNumber(jdata_train_df, jdata_test_df, jdata_user_order_data, cate):
    jdata_train_df_temp = pd.DataFrame()
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLast3MonthBuyNumberMonthly(jdata_train_df[(jdata_train_df.month == 4) & (jdata_train_df.year == 2017)], jdata_user_order_data, datetime.datetime(2017, 1, 1), datetime.datetime(2017, 4, 1), cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLast3MonthBuyNumberMonthly(jdata_train_df[(jdata_train_df.month == 3) & (jdata_train_df.year == 2017)], jdata_user_order_data, datetime.datetime(2016, 12, 1), datetime.datetime(2017, 3, 1), cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLast3MonthBuyNumberMonthly(jdata_train_df[(jdata_train_df.month == 2) & (jdata_train_df.year == 2017)], jdata_user_order_data, datetime.datetime(2016, 11, 1), datetime.datetime(2017, 2, 1), cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLast3MonthBuyNumberMonthly(jdata_train_df[(jdata_train_df.month == 1) & (jdata_train_df.year == 2017)], jdata_user_order_data, datetime.datetime(2016, 10, 1), datetime.datetime(2017, 1, 1), cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLast3MonthBuyNumberMonthly(jdata_train_df[(jdata_train_df.month == 12) & (jdata_train_df.year == 2016)], jdata_user_order_data, datetime.datetime(2016, 9, 1), datetime.datetime(2016, 12, 1), cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLast3MonthBuyNumberMonthly(jdata_train_df[(jdata_train_df.month == 11) & (jdata_train_df.year == 2016)], jdata_user_order_data, datetime.datetime(2016, 8, 1), datetime.datetime(2016, 11, 1), cate)])
    jdata_test_df = getUserCateLast3MonthBuyNumberMonthly(jdata_test_df, jdata_user_order_data, datetime.datetime(2017, 2, 1), datetime.datetime(2017, 5, 1), cate)
    return jdata_train_df_temp, jdata_test_df

print(len(jdata_train_df_huachuang))
print(len(jdata_train_df_huachuang.columns.values))
print(len(jdata_test_df))
print(len(jdata_test_df.columns.values))
jdata_train_df_huachuang, jdata_test_df = getUserCateLast3MonthBuyNumber(jdata_train_df_huachuang, jdata_test_df, jdata_user_order_data, 101)
jdata_train_df_huachuang, jdata_test_df = getUserCateLast3MonthBuyNumber(jdata_train_df_huachuang, jdata_test_df, jdata_user_order_data, 30)
jdata_train_df_huachuang, jdata_test_df = getUserCateLast3MonthBuyNumber(jdata_train_df_huachuang, jdata_test_df, jdata_user_order_data, 1)
jdata_train_df_huachuang, jdata_test_df = getUserCateLast3MonthBuyNumber(jdata_train_df_huachuang, jdata_test_df, jdata_user_order_data, 71)
jdata_train_df_huachuang, jdata_test_df = getUserCateLast3MonthBuyNumber(jdata_train_df_huachuang, jdata_test_df, jdata_user_order_data, 46)
jdata_train_df_huachuang, jdata_test_df = getUserCateLast3MonthBuyNumber(jdata_train_df_huachuang, jdata_test_df, jdata_user_order_data, 83)
print(len(jdata_train_df_huachuang))
print(len(jdata_train_df_huachuang.columns.values))
print(len(jdata_test_df))
print(len(jdata_test_df.columns.values))


593544
101
98924
99
593544
107
98924
105


In [44]:
# 统计某用户上个月下单某个类目商品的次数，逐个月进行处理
def getUserCateLastMonthOrderNumberMonthly(jdata_df, jdata_user_order_data, year, month, cate):
    jdata_train_df_dealMonth = jdata_df
    jdata_user_order_data_buy_future = jdata_user_order_data[(jdata_user_order_data.month == month) & (jdata_user_order_data.year == year) & (jdata_user_order_data.cate == cate)]
    jdata_user_order_data_buy_future_pivot_table = pd.pivot_table(jdata_user_order_data_buy_future, index=['user_id'], values=['o_sku_num'], aggfunc=len)
    jdata_user_order_data_buy_future_pivot_table.reset_index(inplace=True)
    jdata_user_order_data_buy_future_pivot_table.rename(columns={'o_sku_num':'user_cate'+ str(cate) + '_lastMonth_orderNumber'}, inplace=True)
    jdata_train_df_dealMonth = pd.merge(jdata_train_df_dealMonth, jdata_user_order_data_buy_future_pivot_table, on=['user_id'], how='left')
    jdata_train_df_dealMonth['user_cate'+ str(cate) + '_lastMonth_orderNumber'] = jdata_train_df_dealMonth['user_cate'+ str(cate) + '_lastMonth_orderNumber'].fillna(0)
    return jdata_train_df_dealMonth


def getUserCateLastMonthOrderNumber(jdata_train_df, jdata_test_df, jdata_user_order_data, cate):
    jdata_train_df_temp = pd.DataFrame()
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLastMonthOrderNumberMonthly(jdata_train_df[(jdata_train_df.month == 4) & (jdata_train_df.year == 2017)], jdata_user_order_data, 2017, 3, cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLastMonthOrderNumberMonthly(jdata_train_df[(jdata_train_df.month == 3) & (jdata_train_df.year == 2017)], jdata_user_order_data, 2017, 2, cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLastMonthOrderNumberMonthly(jdata_train_df[(jdata_train_df.month == 2) & (jdata_train_df.year == 2017)], jdata_user_order_data, 2017, 1, cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLastMonthOrderNumberMonthly(jdata_train_df[(jdata_train_df.month == 1) & (jdata_train_df.year == 2017)], jdata_user_order_data, 2016, 12, cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLastMonthOrderNumberMonthly(jdata_train_df[(jdata_train_df.month == 12) & (jdata_train_df.year == 2016)], jdata_user_order_data, 2016, 11, cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLastMonthOrderNumberMonthly(jdata_train_df[(jdata_train_df.month == 11) & (jdata_train_df.year == 2016)], jdata_user_order_data, 2016, 10, cate)])
    jdata_test_df = getUserCateLastMonthOrderNumberMonthly(jdata_test_df, jdata_user_order_data, 2017, 4, cate)
    return jdata_train_df_temp, jdata_test_df

print(len(jdata_train_df_huachuang))
print(len(jdata_train_df_huachuang.columns.values))
print(len(jdata_test_df))
print(len(jdata_test_df.columns.values))
jdata_train_df_huachuang, jdata_test_df = getUserCateLastMonthOrderNumber(jdata_train_df_huachuang, jdata_test_df, jdata_user_order_data, 101)
jdata_train_df_huachuang, jdata_test_df = getUserCateLastMonthOrderNumber(jdata_train_df_huachuang, jdata_test_df, jdata_user_order_data, 30)
jdata_train_df_huachuang, jdata_test_df = getUserCateLastMonthOrderNumber(jdata_train_df_huachuang, jdata_test_df, jdata_user_order_data, 1)
jdata_train_df_huachuang, jdata_test_df = getUserCateLastMonthOrderNumber(jdata_train_df_huachuang, jdata_test_df, jdata_user_order_data, 71)
jdata_train_df_huachuang, jdata_test_df = getUserCateLastMonthOrderNumber(jdata_train_df_huachuang, jdata_test_df, jdata_user_order_data, 46)
jdata_train_df_huachuang, jdata_test_df = getUserCateLastMonthOrderNumber(jdata_train_df_huachuang, jdata_test_df, jdata_user_order_data, 83)
print(len(jdata_train_df_huachuang))
print(len(jdata_train_df_huachuang.columns.values))
print(len(jdata_test_df))
print(len(jdata_test_df.columns.values))


593544
107
98924
105
593544
113
98924
111


In [45]:
# 统计某用户上3个月下单某个类目商品的次数，逐个月进行处理
def getUserCateLast3MonthOrderNumberMonthly(jdata_df, jdata_user_order_data, start_time, end_time, cate):
    jdata_train_df_dealMonth = jdata_df
    jdata_user_order_data_buy_future = jdata_user_order_data[(jdata_user_order_data.o_date >= start_time) & (jdata_user_order_data.o_date < end_time) & (jdata_user_order_data.cate == cate)]
    jdata_user_order_data_buy_future_pivot_table = pd.pivot_table(jdata_user_order_data_buy_future, index=['user_id'], values=['o_sku_num'], aggfunc=len)
    jdata_user_order_data_buy_future_pivot_table.reset_index(inplace=True)
    jdata_user_order_data_buy_future_pivot_table.rename(columns={'o_sku_num':'user_cate'+ str(cate) + '_last3Month_orderNumber'}, inplace=True)
    jdata_train_df_dealMonth = pd.merge(jdata_train_df_dealMonth, jdata_user_order_data_buy_future_pivot_table, on=['user_id'], how='left')
    jdata_train_df_dealMonth['user_cate'+ str(cate) + '_last3Month_orderNumber'] = jdata_train_df_dealMonth['user_cate'+ str(cate) + '_last3Month_orderNumber'].fillna(0)
    return jdata_train_df_dealMonth


def getUserCateLast3MonthOrderNumber(jdata_train_df, jdata_test_df, jdata_user_order_data, cate):
    jdata_train_df_temp = pd.DataFrame()
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLast3MonthOrderNumberMonthly(jdata_train_df[(jdata_train_df.month == 4) & (jdata_train_df.year == 2017)], jdata_user_order_data, datetime.datetime(2017, 1, 1), datetime.datetime(2017, 4, 1), cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLast3MonthOrderNumberMonthly(jdata_train_df[(jdata_train_df.month == 3) & (jdata_train_df.year == 2017)], jdata_user_order_data, datetime.datetime(2016, 12, 1), datetime.datetime(2017, 3, 1), cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLast3MonthOrderNumberMonthly(jdata_train_df[(jdata_train_df.month == 2) & (jdata_train_df.year == 2017)], jdata_user_order_data, datetime.datetime(2016, 11, 1), datetime.datetime(2017, 2, 1), cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLast3MonthOrderNumberMonthly(jdata_train_df[(jdata_train_df.month == 1) & (jdata_train_df.year == 2017)], jdata_user_order_data, datetime.datetime(2016, 10, 1), datetime.datetime(2017, 1, 1), cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLast3MonthOrderNumberMonthly(jdata_train_df[(jdata_train_df.month == 12) & (jdata_train_df.year == 2016)], jdata_user_order_data, datetime.datetime(2016, 9, 1), datetime.datetime(2016, 12, 1), cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLast3MonthOrderNumberMonthly(jdata_train_df[(jdata_train_df.month == 11) & (jdata_train_df.year == 2016)], jdata_user_order_data, datetime.datetime(2016, 8, 1), datetime.datetime(2016, 11, 1), cate)])
    jdata_test_df = getUserCateLast3MonthOrderNumberMonthly(jdata_test_df, jdata_user_order_data, datetime.datetime(2017, 2, 1), datetime.datetime(2017, 5, 1), cate)
    return jdata_train_df_temp, jdata_test_df

print(len(jdata_train_df_huachuang))
print(len(jdata_train_df_huachuang.columns.values))
print(len(jdata_test_df))
print(len(jdata_test_df.columns.values))
jdata_train_df_huachuang, jdata_test_df = getUserCateLast3MonthOrderNumber(jdata_train_df_huachuang, jdata_test_df, jdata_user_order_data, 101)
jdata_train_df_huachuang, jdata_test_df = getUserCateLast3MonthOrderNumber(jdata_train_df_huachuang, jdata_test_df, jdata_user_order_data, 30)
jdata_train_df_huachuang, jdata_test_df = getUserCateLast3MonthOrderNumber(jdata_train_df_huachuang, jdata_test_df, jdata_user_order_data, 1)
jdata_train_df_huachuang, jdata_test_df = getUserCateLast3MonthOrderNumber(jdata_train_df_huachuang, jdata_test_df, jdata_user_order_data, 71)
jdata_train_df_huachuang, jdata_test_df = getUserCateLast3MonthOrderNumber(jdata_train_df_huachuang, jdata_test_df, jdata_user_order_data, 46)
jdata_train_df_huachuang, jdata_test_df = getUserCateLast3MonthOrderNumber(jdata_train_df_huachuang, jdata_test_df, jdata_user_order_data, 83)
print(len(jdata_train_df_huachuang))
print(len(jdata_train_df_huachuang.columns.values))
print(len(jdata_test_df))
print(len(jdata_test_df.columns.values))


593544
113
98924
111
593544
119
98924
117


In [46]:
# 统计用户上个月评论的个数
def getUserLastMonthCommentNumberMonthly(jdata_df, jdata_user_comment_score_data, year, month):
    jdata_train_df_dealMonth = jdata_df
    jdata_user_comment_score_data_future = jdata_user_comment_score_data[(jdata_user_comment_score_data.month == month) & (jdata_user_comment_score_data.year == year)]
    jdata_user_comment_score_data_future_pivot_table = pd.pivot_table(jdata_user_comment_score_data_future, index=['user_id'], values=['o_id'], aggfunc=len)
    jdata_user_comment_score_data_future_pivot_table.reset_index(inplace=True)
    jdata_user_comment_score_data_future_pivot_table.rename(columns={'o_id':'user_lastMonth_commentNumber'}, inplace=True)
    jdata_train_df_dealMonth = pd.merge(jdata_train_df_dealMonth, jdata_user_comment_score_data_future_pivot_table, on=['user_id'], how='left')
    jdata_train_df_dealMonth['user_lastMonth_commentNumber'] = jdata_train_df_dealMonth['user_lastMonth_commentNumber'].fillna(0)
    return jdata_train_df_dealMonth

def getUserLastMonthCommentNumber(jdata_train_df, jdata_test_df, jdata_user_comment_score_data):
    jdata_train_df_temp = pd.DataFrame()
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserLastMonthCommentNumberMonthly(jdata_train_df[(jdata_train_df.month == 4) & (jdata_train_df.year == 2017)], jdata_user_comment_score_data, 2017, 3)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserLastMonthCommentNumberMonthly(jdata_train_df[(jdata_train_df.month == 3) & (jdata_train_df.year == 2017)], jdata_user_comment_score_data, 2017, 2)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserLastMonthCommentNumberMonthly(jdata_train_df[(jdata_train_df.month == 2) & (jdata_train_df.year == 2017)], jdata_user_comment_score_data, 2017, 1)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserLastMonthCommentNumberMonthly(jdata_train_df[(jdata_train_df.month == 1) & (jdata_train_df.year == 2017)], jdata_user_comment_score_data, 2016, 12)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserLastMonthCommentNumberMonthly(jdata_train_df[(jdata_train_df.month == 12) & (jdata_train_df.year == 2016)], jdata_user_comment_score_data, 2016, 11)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserLastMonthCommentNumberMonthly(jdata_train_df[(jdata_train_df.month == 11) & (jdata_train_df.year == 2016)], jdata_user_comment_score_data, 2016, 10)])
    jdata_test_df = getUserLastMonthCommentNumberMonthly(jdata_test_df, jdata_user_comment_score_data, 2017, 4)
    return jdata_train_df_temp, jdata_test_df

print(len(jdata_train_df_huachuang))
print(len(jdata_train_df_huachuang.columns.values))
print(len(jdata_test_df))
print(len(jdata_test_df.columns.values))
jdata_train_df_huachuang, jdata_test_df = getUserLastMonthCommentNumber(jdata_train_df_huachuang, jdata_test_df, jdata_user_comment_score_data)
print(len(jdata_train_df_huachuang))
print(len(jdata_train_df_huachuang.columns.values))
print(len(jdata_test_df))
print(len(jdata_test_df.columns.values))


593544
119
98924
117
593544
120
98924
118


In [47]:
# 统计用户上3个月评论的个数
def getUserLast3MonthCommentNumberMonthly(jdata_df, jdata_user_comment_score_data, start_time, end_time):
    jdata_train_df_dealMonth = jdata_df
    jdata_user_comment_score_data_future = jdata_user_comment_score_data[(jdata_user_comment_score_data.comment_create_tm >= start_time) & (jdata_user_comment_score_data.comment_create_tm < end_time)]
    jdata_user_comment_score_data_future_pivot_table = pd.pivot_table(jdata_user_comment_score_data_future, index=['user_id'], values=['o_id'], aggfunc=len)
    jdata_user_comment_score_data_future_pivot_table.reset_index(inplace=True)
    jdata_user_comment_score_data_future_pivot_table.rename(columns={'o_id':'user_last3Month_commentNumber'}, inplace=True)
    jdata_train_df_dealMonth = pd.merge(jdata_train_df_dealMonth, jdata_user_comment_score_data_future_pivot_table, on=['user_id'], how='left')
    jdata_train_df_dealMonth['user_last3Month_commentNumber'] = jdata_train_df_dealMonth['user_last3Month_commentNumber'].fillna(0)
    return jdata_train_df_dealMonth

def getUserLast3MonthCommentNumber(jdata_train_df, jdata_test_df, jdata_user_comment_score_data):
    jdata_train_df_temp = pd.DataFrame()
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserLast3MonthCommentNumberMonthly(jdata_train_df[(jdata_train_df.month == 4) & (jdata_train_df.year == 2017)], jdata_user_comment_score_data, datetime.datetime(2017, 1, 1), datetime.datetime(2017, 4, 1))])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserLast3MonthCommentNumberMonthly(jdata_train_df[(jdata_train_df.month == 3) & (jdata_train_df.year == 2017)], jdata_user_comment_score_data, datetime.datetime(2016, 12, 1), datetime.datetime(2017, 3, 1))])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserLast3MonthCommentNumberMonthly(jdata_train_df[(jdata_train_df.month == 2) & (jdata_train_df.year == 2017)], jdata_user_comment_score_data, datetime.datetime(2016, 11, 1), datetime.datetime(2017, 2, 1))])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserLast3MonthCommentNumberMonthly(jdata_train_df[(jdata_train_df.month == 1) & (jdata_train_df.year == 2017)], jdata_user_comment_score_data, datetime.datetime(2016, 10, 1), datetime.datetime(2017, 1, 1))])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserLast3MonthCommentNumberMonthly(jdata_train_df[(jdata_train_df.month == 12) & (jdata_train_df.year == 2016)], jdata_user_comment_score_data, datetime.datetime(2016, 9, 1), datetime.datetime(2016, 12, 1))])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserLast3MonthCommentNumberMonthly(jdata_train_df[(jdata_train_df.month == 11) & (jdata_train_df.year == 2016)], jdata_user_comment_score_data, datetime.datetime(2016, 8, 1), datetime.datetime(2016, 11, 1))])
    jdata_test_df = getUserLast3MonthCommentNumberMonthly(jdata_test_df, jdata_user_comment_score_data, datetime.datetime(2017, 2, 1), datetime.datetime(2017, 5, 1))
    return jdata_train_df_temp, jdata_test_df

print(len(jdata_train_df_huachuang))
print(len(jdata_train_df_huachuang.columns.values))
print(len(jdata_test_df))
print(len(jdata_test_df.columns.values))
jdata_train_df_huachuang, jdata_test_df = getUserLast3MonthCommentNumber(jdata_train_df_huachuang, jdata_test_df, jdata_user_comment_score_data)
print(len(jdata_train_df_huachuang))
print(len(jdata_train_df_huachuang.columns.values))
print(len(jdata_test_df))
print(len(jdata_test_df.columns.values))


593544
120
98924
118
593544
121
98924
119


In [48]:
# 统计用户上个月好评，中评，差评评论的个数
def getUserLastMonthScoreCommentNumberMonthly(jdata_df, jdata_user_comment_score_data, year, month, score_level):
    jdata_train_df_dealMonth = jdata_df
    jdata_user_comment_score_data_future = jdata_user_comment_score_data[(jdata_user_comment_score_data.month == month) & (jdata_user_comment_score_data.year == year) & (jdata_user_comment_score_data.score_level == score_level)]
    jdata_user_comment_score_data_future_pivot_table = pd.pivot_table(jdata_user_comment_score_data_future, index=['user_id'], values=['o_id'], aggfunc=len)
    jdata_user_comment_score_data_future_pivot_table.reset_index(inplace=True)
    jdata_user_comment_score_data_future_pivot_table.rename(columns={'o_id':'user_lastMonth_' + str(score_level) + 'commentNumber'}, inplace=True)
    jdata_train_df_dealMonth = pd.merge(jdata_train_df_dealMonth, jdata_user_comment_score_data_future_pivot_table, on=['user_id'], how='left')
    jdata_train_df_dealMonth['user_lastMonth_' + str(score_level) + 'commentNumber'] = jdata_train_df_dealMonth['user_lastMonth_' + str(score_level) + 'commentNumber'].fillna(0)
    return jdata_train_df_dealMonth

def getUserLastMonthScoreCommentNumber(jdata_train_df, jdata_test_df, jdata_user_comment_score_data, score_level):
    jdata_train_df_temp = pd.DataFrame()
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserLastMonthScoreCommentNumberMonthly(jdata_train_df[(jdata_train_df.month == 4) & (jdata_train_df.year == 2017)], jdata_user_comment_score_data, 2017, 3, score_level)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserLastMonthScoreCommentNumberMonthly(jdata_train_df[(jdata_train_df.month == 3) & (jdata_train_df.year == 2017)], jdata_user_comment_score_data, 2017, 2, score_level)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserLastMonthScoreCommentNumberMonthly(jdata_train_df[(jdata_train_df.month == 2) & (jdata_train_df.year == 2017)], jdata_user_comment_score_data, 2017, 1, score_level)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserLastMonthScoreCommentNumberMonthly(jdata_train_df[(jdata_train_df.month == 1) & (jdata_train_df.year == 2017)], jdata_user_comment_score_data, 2016, 12, score_level)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserLastMonthScoreCommentNumberMonthly(jdata_train_df[(jdata_train_df.month == 12) & (jdata_train_df.year == 2016)], jdata_user_comment_score_data, 2016, 11, score_level)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserLastMonthScoreCommentNumberMonthly(jdata_train_df[(jdata_train_df.month == 11) & (jdata_train_df.year == 2016)], jdata_user_comment_score_data, 2016, 10, score_level)])
    jdata_test_df = getUserLastMonthScoreCommentNumberMonthly(jdata_test_df, jdata_user_comment_score_data, 2017, 4, score_level)
    return jdata_train_df_temp, jdata_test_df

print(len(jdata_train_df_huachuang))
print(len(jdata_train_df_huachuang.columns.values))
print(len(jdata_test_df))
print(len(jdata_test_df.columns.values))
jdata_train_df_huachuang, jdata_test_df = getUserLastMonthScoreCommentNumber(jdata_train_df_huachuang, jdata_test_df, jdata_user_comment_score_data, 1)
jdata_train_df_huachuang, jdata_test_df = getUserLastMonthScoreCommentNumber(jdata_train_df_huachuang, jdata_test_df, jdata_user_comment_score_data, 2)
jdata_train_df_huachuang, jdata_test_df = getUserLastMonthScoreCommentNumber(jdata_train_df_huachuang, jdata_test_df, jdata_user_comment_score_data, 3)
print(len(jdata_train_df_huachuang))
print(len(jdata_train_df_huachuang.columns.values))
print(len(jdata_test_df))
print(len(jdata_test_df.columns.values))


593544
121
98924
119
593544
124
98924
122


In [49]:
# 统计用户上3个月好评，中评，差评评论的个数
def getUserLast3MonthScoreCommentNumberMonthly(jdata_df, jdata_user_comment_score_data, start_time, end_time, score_level):
    jdata_train_df_dealMonth = jdata_df
    jdata_user_comment_score_data_future = jdata_user_comment_score_data[(jdata_user_comment_score_data.comment_create_tm >= start_time) & (jdata_user_comment_score_data.comment_create_tm < end_time) & (jdata_user_comment_score_data.score_level == score_level)]
    jdata_user_comment_score_data_future_pivot_table = pd.pivot_table(jdata_user_comment_score_data_future, index=['user_id'], values=['o_id'], aggfunc=len)
    jdata_user_comment_score_data_future_pivot_table.reset_index(inplace=True)
    jdata_user_comment_score_data_future_pivot_table.rename(columns={'o_id':'user_last3Month_' + str(score_level) + 'commentNumber'}, inplace=True)
    jdata_train_df_dealMonth = pd.merge(jdata_train_df_dealMonth, jdata_user_comment_score_data_future_pivot_table, on=['user_id'], how='left')
    jdata_train_df_dealMonth['user_last3Month_' + str(score_level) + 'commentNumber'] = jdata_train_df_dealMonth['user_last3Month_' + str(score_level) + 'commentNumber'].fillna(0)
    return jdata_train_df_dealMonth

def getUserLast3MonthScoreCommentNumber(jdata_train_df, jdata_test_df, jdata_user_comment_score_data, score_level):
    jdata_train_df_temp = pd.DataFrame()
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserLast3MonthScoreCommentNumberMonthly(jdata_train_df[(jdata_train_df.month == 4) & (jdata_train_df.year == 2017)], jdata_user_comment_score_data, datetime.datetime(2017, 1, 1), datetime.datetime(2017, 4, 1), score_level)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserLast3MonthScoreCommentNumberMonthly(jdata_train_df[(jdata_train_df.month == 3) & (jdata_train_df.year == 2017)], jdata_user_comment_score_data, datetime.datetime(2016, 12, 1), datetime.datetime(2017, 3, 1), score_level)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserLast3MonthScoreCommentNumberMonthly(jdata_train_df[(jdata_train_df.month == 2) & (jdata_train_df.year == 2017)], jdata_user_comment_score_data, datetime.datetime(2016, 11, 1), datetime.datetime(2017, 2, 1), score_level)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserLast3MonthScoreCommentNumberMonthly(jdata_train_df[(jdata_train_df.month == 1) & (jdata_train_df.year == 2017)], jdata_user_comment_score_data, datetime.datetime(2016, 10, 1), datetime.datetime(2017, 1, 1), score_level)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserLast3MonthScoreCommentNumberMonthly(jdata_train_df[(jdata_train_df.month == 12) & (jdata_train_df.year == 2016)], jdata_user_comment_score_data, datetime.datetime(2016, 9, 1), datetime.datetime(2016, 12, 1), score_level)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserLast3MonthScoreCommentNumberMonthly(jdata_train_df[(jdata_train_df.month == 11) & (jdata_train_df.year == 2016)], jdata_user_comment_score_data, datetime.datetime(2016, 8, 1), datetime.datetime(2016, 11, 1), score_level)])
    jdata_test_df = getUserLast3MonthScoreCommentNumberMonthly(jdata_test_df, jdata_user_comment_score_data, datetime.datetime(2017, 2, 1), datetime.datetime(2017, 5, 1), score_level)
    return jdata_train_df_temp, jdata_test_df

print(len(jdata_train_df_huachuang))
print(len(jdata_train_df_huachuang.columns.values))
print(len(jdata_test_df))
print(len(jdata_test_df.columns.values))
jdata_train_df_huachuang, jdata_test_df = getUserLast3MonthScoreCommentNumber(jdata_train_df_huachuang, jdata_test_df, jdata_user_comment_score_data, 1)
jdata_train_df_huachuang, jdata_test_df = getUserLast3MonthScoreCommentNumber(jdata_train_df_huachuang, jdata_test_df, jdata_user_comment_score_data, 2)
jdata_train_df_huachuang, jdata_test_df = getUserLast3MonthScoreCommentNumber(jdata_train_df_huachuang, jdata_test_df, jdata_user_comment_score_data, 3)
print(len(jdata_train_df_huachuang))
print(len(jdata_train_df_huachuang.columns.values))
print(len(jdata_test_df))
print(len(jdata_test_df.columns.values))


593544
124
98924
122
593544
127
98924
125


In [50]:
# 统计某用户上个月购买某个类目商品在当月的天数，逐个月进行处理
def getUserCateLastMonthBuyDayMonthly(jdata_df, jdata_user_order_data, year, month, cate):
    jdata_train_df_dealMonth = jdata_df.copy()
    jdata_user_order_data_buy_future = jdata_user_order_data[['user_id', 'day']][(jdata_user_order_data.month == month) & (jdata_user_order_data.year == year) & (jdata_user_order_data.cate == cate)]
    jdata_user_order_data_buy_future = jdata_user_order_data_buy_future.sort_values(by=['user_id', 'day'], ascending=False)
    jdata_user_order_data_buy_future.drop_duplicates(['user_id'], inplace=True)
    jdata_user_order_data_buy_future.rename(columns={'day':'user_cate'+ str(cate) + '_lastMonth_buyDay'}, inplace=True)
    jdata_train_df_dealMonth = pd.merge(jdata_train_df_dealMonth, jdata_user_order_data_buy_future, on=['user_id'], how='left')
    return jdata_train_df_dealMonth

def getUserCateLastMonthBuyDay(jdata_train_df, jdata_test_df, jdata_user_order_data, cate):
    jdata_train_df_temp = pd.DataFrame()
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLastMonthBuyDayMonthly(jdata_train_df[(jdata_train_df.month == 4) & (jdata_train_df.year == 2017)], jdata_user_order_data, 2017, 3, cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLastMonthBuyDayMonthly(jdata_train_df[(jdata_train_df.month == 3) & (jdata_train_df.year == 2017)], jdata_user_order_data, 2017, 2, cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLastMonthBuyDayMonthly(jdata_train_df[(jdata_train_df.month == 2) & (jdata_train_df.year == 2017)], jdata_user_order_data, 2017, 1, cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLastMonthBuyDayMonthly(jdata_train_df[(jdata_train_df.month == 1) & (jdata_train_df.year == 2017)], jdata_user_order_data, 2016, 12, cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLastMonthBuyDayMonthly(jdata_train_df[(jdata_train_df.month == 12) & (jdata_train_df.year == 2016)], jdata_user_order_data, 2016, 11, cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLastMonthBuyDayMonthly(jdata_train_df[(jdata_train_df.month == 11) & (jdata_train_df.year == 2016)], jdata_user_order_data, 2016, 10, cate)])
    jdata_test_df = getUserCateLastMonthBuyDayMonthly(jdata_test_df, jdata_user_order_data, 2017, 4, cate)
    return jdata_train_df_temp, jdata_test_df

print(len(jdata_train_df_huachuang))
print(len(jdata_train_df_huachuang.columns.values))
print(len(jdata_test_df))
print(len(jdata_test_df.columns.values))
jdata_train_df_huachuang, jdata_test_df = getUserCateLastMonthBuyDay(jdata_train_df_huachuang, jdata_test_df, jdata_user_order_data, 101)
jdata_train_df_huachuang, jdata_test_df = getUserCateLastMonthBuyDay(jdata_train_df_huachuang, jdata_test_df, jdata_user_order_data, 30)
jdata_train_df_huachuang, jdata_test_df = getUserCateLastMonthBuyDay(jdata_train_df_huachuang, jdata_test_df, jdata_user_order_data, 1)
jdata_train_df_huachuang, jdata_test_df = getUserCateLastMonthBuyDay(jdata_train_df_huachuang, jdata_test_df, jdata_user_order_data, 71)
jdata_train_df_huachuang, jdata_test_df = getUserCateLastMonthBuyDay(jdata_train_df_huachuang, jdata_test_df, jdata_user_order_data, 46)
jdata_train_df_huachuang, jdata_test_df = getUserCateLastMonthBuyDay(jdata_train_df_huachuang, jdata_test_df, jdata_user_order_data, 83)
print(len(jdata_train_df_huachuang))
print(len(jdata_train_df_huachuang.columns.values))
print(len(jdata_test_df))
print(len(jdata_test_df.columns.values))


593544
127
98924
125
593544
133
98924
131


In [51]:
# 统计某用户上3个月购买某个类目商品在当月的天数，逐个月进行处理
def getUserCateLast3MonthBuyDayMonthly(jdata_df, jdata_user_order_data, start_time, end_time, cate):
    jdata_train_df_dealMonth = jdata_df.copy()
    jdata_user_order_data_buy_future = jdata_user_order_data[['user_id', 'day']][(jdata_user_order_data.o_date >= start_time) & (jdata_user_order_data.o_date < end_time) & (jdata_user_order_data.cate == cate)]
    jdata_user_order_data_buy_future = jdata_user_order_data_buy_future.sort_values(by=['user_id', 'day'], ascending=False)
    jdata_user_order_data_buy_future.drop_duplicates(['user_id'], inplace=True)
    jdata_user_order_data_buy_future.rename(columns={'day':'user_cate'+ str(cate) + '_last3Month_buyDay'}, inplace=True)
    jdata_train_df_dealMonth = pd.merge(jdata_train_df_dealMonth, jdata_user_order_data_buy_future, on=['user_id'], how='left')
    return jdata_train_df_dealMonth

def getUserCateLast3MonthBuyDay(jdata_train_df, jdata_test_df, jdata_user_order_data, cate):
    jdata_train_df_temp = pd.DataFrame()
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLast3MonthBuyDayMonthly(jdata_train_df[(jdata_train_df.month == 4) & (jdata_train_df.year == 2017)], jdata_user_order_data, datetime.datetime(2017, 1, 1), datetime.datetime(2017, 4, 1), cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLast3MonthBuyDayMonthly(jdata_train_df[(jdata_train_df.month == 3) & (jdata_train_df.year == 2017)], jdata_user_order_data, datetime.datetime(2016, 12, 1), datetime.datetime(2017, 3, 1), cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLast3MonthBuyDayMonthly(jdata_train_df[(jdata_train_df.month == 2) & (jdata_train_df.year == 2017)], jdata_user_order_data, datetime.datetime(2016, 11, 1), datetime.datetime(2017, 2, 1), cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLast3MonthBuyDayMonthly(jdata_train_df[(jdata_train_df.month == 1) & (jdata_train_df.year == 2017)], jdata_user_order_data, datetime.datetime(2016, 10, 1), datetime.datetime(2017, 1, 1), cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLast3MonthBuyDayMonthly(jdata_train_df[(jdata_train_df.month == 12) & (jdata_train_df.year == 2016)], jdata_user_order_data, datetime.datetime(2016, 9, 1), datetime.datetime(2016, 12, 1), cate)])
    jdata_train_df_temp = pd.concat([jdata_train_df_temp, getUserCateLast3MonthBuyDayMonthly(jdata_train_df[(jdata_train_df.month == 11) & (jdata_train_df.year == 2016)], jdata_user_order_data, datetime.datetime(2016, 8, 1), datetime.datetime(2016, 11, 1), cate)])
    jdata_test_df = getUserCateLast3MonthBuyDayMonthly(jdata_test_df, jdata_user_order_data, datetime.datetime(2017, 2, 1), datetime.datetime(2017, 5, 1), cate)
    return jdata_train_df_temp, jdata_test_df

print(len(jdata_train_df_huachuang))
print(len(jdata_train_df_huachuang.columns.values))
print(len(jdata_test_df))
print(len(jdata_test_df.columns.values))
jdata_train_df_huachuang, jdata_test_df = getUserCateLast3MonthBuyDay(jdata_train_df_huachuang, jdata_test_df, jdata_user_order_data, 101)
jdata_train_df_huachuang, jdata_test_df = getUserCateLast3MonthBuyDay(jdata_train_df_huachuang, jdata_test_df, jdata_user_order_data, 30)
jdata_train_df_huachuang, jdata_test_df = getUserCateLast3MonthBuyDay(jdata_train_df_huachuang, jdata_test_df, jdata_user_order_data, 1)
jdata_train_df_huachuang, jdata_test_df = getUserCateLast3MonthBuyDay(jdata_train_df_huachuang, jdata_test_df, jdata_user_order_data, 71)
jdata_train_df_huachuang, jdata_test_df = getUserCateLast3MonthBuyDay(jdata_train_df_huachuang, jdata_test_df, jdata_user_order_data, 46)
jdata_train_df_huachuang, jdata_test_df = getUserCateLast3MonthBuyDay(jdata_train_df_huachuang, jdata_test_df, jdata_user_order_data, 83)
print(len(jdata_train_df_huachuang))
print(len(jdata_train_df_huachuang.columns.values))
print(len(jdata_test_df))
print(len(jdata_test_df.columns.values))


593544
133
98924
131
593544
139
98924
137


In [52]:
print(len(jdata_train_df_huachuang.columns.values))
print(jdata_train_df_huachuang.columns.values)

print(len(jdata_test_df.columns.values))
print(jdata_test_df.columns.values)

print(jdata_train_df_huachuang.head(10))


139
['user_id' 'age' 'sex' 'user_lv_cd' 'month' 'year' 'is_order' 'order_day'
 'last_buy_cate101_deltaDay' 'last_buy_cate30_deltaDay'
 'last_buy_cate1_deltaDay' 'last_buy_cate71_deltaDay'
 'last_buy_cate46_deltaDay' 'last_buy_cate83_deltaDay'
 'last_buy_targetCate_deltaDay' 'last_buy_relatedCate_deltaDay'
 'last_focus_cate101_number' 'last_focus_cate30_number'
 'last_focus_cate1_number' 'last_focus_cate71_number'
 'last_focus_cate46_number' 'last_focus_cate83_number'
 'last_focus_targetCate_number' 'last_focus_relatedCate_number'
 'last_click_cate101_deltaDay' 'last_click_cate30_deltaDay'
 'last_click_cate1_deltaDay' 'last_click_cate71_deltaDay'
 'last_click_cate46_deltaDay' 'last_click_cate83_deltaDay'
 'last_click_TargetCate_deltaDay' 'last_click_relatedCate_deltaDay'
 'last_buy_cate101_number' 'last_buy_cate30_number' 'last_buy_cate1_number'
 'last_buy_cate71_number' 'last_buy_cate46_number' 'last_buy_cate83_number'
 'last_buy_targetCate_number' 'last_buy_relatedCate_number'
 'user_

In [53]:
print(jdata_train_df_huachuang.info())
print(jdata_test_df.info())


<class 'pandas.core.frame.DataFrame'>
Int64Index: 593544 entries, 0 to 98923
Columns: 139 entries, user_id to user_cate83_last3Month_buyDay
dtypes: float64(116), int64(23)
memory usage: 634.0 MB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 98924 entries, 0 to 98923
Columns: 137 entries, user_id to user_cate83_last3Month_buyDay
dtypes: float64(115), int64(22)
memory usage: 104.2 MB
None


In [54]:
# 导出训练集和测试集预处理结果
def exportResult(df, fileName):
    df.to_csv('./%s.csv' % fileName, header=True, index=False)
    
exportResult(jdata_train_df_huachuang, 'chusai_jdata_train_df_huachuang')
exportResult(jdata_test_df, 'chusai_jdata_test_df_huachuang')
